In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, Dataset
from sklearn.metrics import precision_score, recall_score, f1_score

from PIL import Image
import io, pickle
from tqdm import tqdm, trange
import numpy as np
import random

/opt/conda/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
with open('images/images.pickle', 'rb') as f:
    img_data = pickle.load(f)
class CustomDataset(Dataset):
    def __init__(self, img_data, txt_file, transform=None):
        self.data = img_data
        with open(txt_file, 'r') as f:
            self.labels = f.readlines()
        self.transform = transform
    def __len__(self):
        return len(self.labels)
    def __getitem__(self, idx):
        img_path, label = self.labels[idx].strip().split()
        img_path = img_path.split('/')
        img = self.data[img_path[1]][img_path[2]]
        img = Image.open(io.BytesIO(img)).convert('RGB')
        if self.transform:
            img = self.transform(img)
        label = torch.tensor(int(label))
        return img, label


transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])
dataset_train = CustomDataset(img_data=img_data, txt_file='images/train.txt', transform=transform)
dataset_val = CustomDataset(img_data=img_data, txt_file='images/val.txt', transform=transform)
dataset_test = CustomDataset(img_data=img_data, txt_file='images/test.txt', transform=transform)


data_loader_train = DataLoader(dataset_train, batch_size=32, shuffle=True, )
data_loader_val = DataLoader(dataset_val, batch_size=32, shuffle=False)
data_loader_test = DataLoader(dataset_test, batch_size=32, shuffle=False)


In [9]:
if torch.cuda.is_available():
    device = torch.device("cuda")
else:
    device = torch.device("cpu")
class SelfAttention(nn.Module):
    def __init__(self, in_channels):
        super(SelfAttention, self).__init__()
        self.query_conv = nn.Conv2d(in_channels, in_channels // 8, kernel_size=1)
        self.key_conv = nn.Conv2d(in_channels, in_channels // 8, kernel_size=1)
        self.value_conv = nn.Conv2d(in_channels, in_channels, kernel_size=1)
        self.gamma = nn.Parameter(torch.zeros(1))
        
    def forward(self, x):
        batch_size, C, width, height = x.size()
        
        # 计算查询、键和值
        proj_query = self.query_conv(x).view(batch_size, -1, width * height).permute(0, 2, 1)
        proj_key = self.key_conv(x).view(batch_size, -1, width * height)
        energy = torch.bmm(proj_query, proj_key)
        
        # 计算注意力权重并应用到值上
        attention = F.softmax(energy, dim=-1)
        proj_value = self.value_conv(x).view(batch_size, -1, width * height)
        out = torch.bmm(proj_value, attention.permute(0, 2, 1))
        
        # 调整输出并加上残差连接
        out = out.view(batch_size, C, width, height)
        out = self.gamma * out + x
        return out

class CNNModel(nn.Module):
    def __init__(self, num_classes=50):
        super(CNNModel, self).__init__()
        self.conv1 = nn.Conv2d(3, 64, kernel_size=3, padding=1)
        self.bn1 = nn.BatchNorm2d(64)
        self.conv2 = nn.Conv2d(64, 128, kernel_size=3, padding=1)
        self.bn2 = nn.BatchNorm2d(128)
        self.attention1 = SelfAttention(128)

        self.fc1 = nn.Linear(128*56*56, 1024)
        # self.fc2 = nn.Linear(4096, 512)
        # self.fc3 = nn.Linear(512, num_classes)
        self.fc2 = nn.Linear(1024, num_classes)
        self.activation = nn.ReLU()
        self.flatten = nn.Flatten()
        self.pool = nn.MaxPool2d(2, 2)
        # self.dropout25 = nn.Dropout(0.25)
        self.dropout50 = nn.Dropout(0.5)
        
    def forward(self, x):
        x = self.pool(self.activation(self.bn1(self.conv1(x))))
        x = self.pool(self.activation(self.bn2(self.conv2(x))))
        x = self.attention1(x)
        x = self.dropout50(self.flatten(x))
        x = self.dropout50(self.activation(self.fc1(x)))
        # x = self.dropout50(self.activation(self.fc2(x)))
        # x = self.fc3(x)
        x = self.fc2(x)
        return x


for lr in [1e-6]:
    model = CNNModel(num_classes=50).to(device)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=lr)
    num_epochs = 10
    save_path = 'record/Q2_less_layers'
    model_path = f'{save_path}/model_weight/model_weight_lr{lr}_bn_dropout50.pth'
    best_val_loss = float('inf')
    best_model = None
    train_loss = []
    train_acc = []
    val_loss = []
    val_acc = []
    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0
        running_acc = 0.0
        pbar = tqdm(data_loader_train, colour='green', total=len(data_loader_train))
        for images, labels in pbar:
            images, labels = images.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()*len(labels)
            running_acc += (outputs.argmax(dim=1) == labels).float().sum().item()
            pbar.set_description(f'Train Iter: {epoch+1:03}/{num_epochs:03}  Loss: {loss:3.4f}')
            pbar.update()
        running_loss_avg = running_loss / len(dataset_train)
        running_acc_avg = running_acc / len(dataset_train)
        train_loss.append(running_loss_avg)
        train_acc.append(running_acc_avg)
        print(f"Epoch {epoch+1}, Loss: {running_loss_avg}, Accuracy: {running_acc_avg}")

        model.eval()
        with torch.no_grad():
            class_counts = [0] * 50
            running_loss = 0.0
            running_acc = 0.0
            pbar = tqdm(data_loader_val, colour='red', total=len(data_loader_val))
            for images, labels in pbar:
                images, labels = images.to(device), labels.to(device)
                outputs = model(images)
                predictions = outputs.argmax(dim=1)
                loss = criterion(outputs, labels)
                running_loss += loss.item()*len(labels)
                running_acc += (predictions == labels).float().sum().item()
                pbar.set_description(f'Valid Iter: {epoch+1:03}/{num_epochs:03}  Loss: {loss:3.4f}')
                pbar.update()

                for pred_label in predictions:
                    class_counts[pred_label] += 1

            running_loss_avg = running_loss / len(dataset_val)
            running_acc_avg = running_acc / len(dataset_val)
            val_loss.append(running_loss_avg)
            val_acc.append(running_acc_avg)
            print(f"Validation Loss: {running_loss_avg}, Accuracy: {running_acc_avg}")
            print(f"Class Counts: {class_counts}")
            if running_loss_avg < best_val_loss:
                best_val_loss = running_loss
                best_model = model
                torch.save(model.state_dict(), model_path)
        print("======================================")
    print("Finished Training")


    model.eval()
    with torch.no_grad():
        class_counts = [0] * 50
        running_loss = 0.0
        running_acc = 0.0
        predictions = []
        true_labels = []
        pbar = tqdm(data_loader_test, colour='red', total=len(data_loader_test))
        for images, labels in pbar:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)

            _, predicted = torch.max(outputs, 1)
            predictions.extend(predicted.cpu().numpy())
            true_labels.extend(labels.cpu().numpy())
            loss = criterion(outputs, labels)
            running_loss += loss.item()*len(labels)
            running_acc += (outputs.argmax(dim=1) == labels).float().sum().item()
            pbar.set_description(f'Test Iter: {epoch+1:03}/{num_epochs:03}  Loss: {loss:3.4f}')
            pbar.update()

        for pred_label in predictions:
            class_counts[pred_label] += 1
        
        precision = precision_score(true_labels, predictions, average='weighted')
        recall = recall_score(true_labels, predictions, average='weighted')
        f1 = f1_score(true_labels, predictions, average='weighted')
        print(f"Test Loss: {running_loss/len(dataset_test)}, Accuracy: {running_acc/len(dataset_test)}")
        print(f"Precision: {precision}, Recall: {recall}, F1: {f1}")
        print(f"Class Counts: {class_counts}")
    print("Finished Testing")

    train_record = {"train_loss": train_loss, "train_acc": train_acc, "val_loss": val_loss, "val_acc": val_acc}
    test_record = {"true_labels": true_labels, "predictions": predictions, "precision": precision, "recall": recall, "f1": f1, "class_counts": class_counts}


    with open(f'{save_path}/train_record/train_record_lr{lr}_bn_dropout50.pkl', 'wb') as f:
        pickle.dump(train_record, f)
    with open(f'{save_path}/test_record/test_record_lr{lr}_bn_dropout50.pkl', 'wb') as f:
        pickle.dump(test_record, f)


Train Iter: 001/010  Loss: 3.2567: 100%|██████████| 1979/1979 [04:23<00:00,  7.51it/s]


Epoch 1, Loss: 3.5076264987159425, Accuracy: 0.10776154757204895


Valid Iter: 001/010  Loss: 2.4816: 100%|██████████| 15/15 [00:00<00:00, 22.42it/s]


Validation Loss: 3.167535139719645, Accuracy: 0.1622222222222222
Class Counts: [12, 8, 44, 9, 23, 5, 6, 23, 1, 1, 0, 1, 0, 5, 3, 5, 0, 4, 13, 27, 13, 8, 4, 7, 7, 0, 0, 14, 1, 5, 2, 0, 4, 24, 13, 1, 25, 7, 3, 8, 13, 10, 14, 2, 2, 12, 37, 3, 16, 5]


Train Iter: 002/010  Loss: 3.2722: 100%|██████████| 1979/1979 [04:24<00:00,  7.47it/s]


...

Valid Iter: 010/010  Loss: 1.1329: 100%|██████████| 15/15 [00:00<00:00, 22.26it/s]


Validation Loss: 2.725719035466512, Accuracy: 0.24888888888888888
Class Counts: [5, 2, 17, 4, 22, 16, 8, 10, 7, 7, 3, 0, 5, 8, 11, 8, 3, 5, 14, 5, 32, 4, 7, 8, 3, 9, 11, 2, 18, 2, 4, 14, 13, 8, 9, 3, 7, 15, 6, 11, 16, 11, 3, 19, 7, 12, 11, 2, 12, 11]
Finished Training


Test Iter: 010/010  Loss: 1.1329: 100%|██████████| 15/15 [00:00<00:00, 22.40it/s]


Test Loss: 1.1328809261322021, Accuracy: 0.28
Precision: 0.29624302878139197, Recall: 0.28, F1: 0.262235022776772
Class Counts: [11, 3, 15, 4, 9, 20, 8, 15, 2, 6, 1, 0, 3, 7, 10, 15, 1, 4, 17, 4, 25, 2, 12, 13, 3, 8, 11, 3, 23, 3, 3, 10, 8, 21, 11, 1, 3, 11, 7, 7, 18, 14, 5, 11, 10, 12, 14, 5, 9, 12]
Finished Testing


/opt/conda/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [10]:
if torch.cuda.is_available():
    device = torch.device("cuda")
else:
    device = torch.device("cpu")
class SelfAttention(nn.Module):
    def __init__(self, in_channels):
        super(SelfAttention, self).__init__()
        self.query_conv = nn.Conv2d(in_channels, in_channels // 8, kernel_size=1)
        self.key_conv = nn.Conv2d(in_channels, in_channels // 8, kernel_size=1)
        self.value_conv = nn.Conv2d(in_channels, in_channels, kernel_size=1)
        self.gamma = nn.Parameter(torch.zeros(1))
        
    def forward(self, x):
        batch_size, C, width, height = x.size()
        
        # 计算查询、键和值
        proj_query = self.query_conv(x).view(batch_size, -1, width * height).permute(0, 2, 1)
        proj_key = self.key_conv(x).view(batch_size, -1, width * height)
        energy = torch.bmm(proj_query, proj_key)
        
        # 计算注意力权重并应用到值上
        attention = F.softmax(energy, dim=-1)
        proj_value = self.value_conv(x).view(batch_size, -1, width * height)
        out = torch.bmm(proj_value, attention.permute(0, 2, 1))
        
        # 调整输出并加上残差连接
        out = out.view(batch_size, C, width, height)
        out = self.gamma * out + x
        return out

class CNNModel(nn.Module):
    def __init__(self, num_classes=50):
        super(CNNModel, self).__init__()
        self.conv1 = nn.Conv2d(3, 64, kernel_size=3, padding=1)
        self.bn1 = nn.BatchNorm2d(64)
        self.conv2 = nn.Conv2d(64, 128, kernel_size=3, padding=1)
        self.bn2 = nn.BatchNorm2d(128)
        self.attention1 = SelfAttention(128)

        self.fc1 = nn.Linear(128*56*56, 1024)
        # self.fc2 = nn.Linear(4096, 512)
        # self.fc3 = nn.Linear(512, num_classes)
        self.fc2 = nn.Linear(1024, num_classes)
        self.activation = nn.ReLU()
        self.flatten = nn.Flatten()
        self.pool = nn.MaxPool2d(2, 2)
        # self.dropout25 = nn.Dropout(0.25)
        self.dropout50 = nn.Dropout(0.5)
        
    def forward(self, x):
        x = self.pool(self.activation(self.bn1(self.conv1(x))))
        x = self.pool(self.activation(self.bn2(self.conv2(x))))
        x = self.attention1(x)
        x = self.dropout50(self.flatten(x))
        x = self.dropout50(self.activation(self.fc1(x)))
        # x = self.dropout50(self.activation(self.fc2(x)))
        # x = self.fc3(x)
        x = self.fc2(x)
        return x


for lr in [1e-5]:
    model = CNNModel(num_classes=50).to(device)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=lr)
    num_epochs = 10
    save_path = 'record/Q2_less_layers'
    model_path = f'{save_path}/model_weight/model_weight_lr{lr}_bn_dropout50.pth'
    best_val_loss = float('inf')
    best_model = None
    train_loss = []
    train_acc = []
    val_loss = []
    val_acc = []
    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0
        running_acc = 0.0
        pbar = tqdm(data_loader_train, colour='green', total=len(data_loader_train))
        for images, labels in pbar:
            images, labels = images.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()*len(labels)
            running_acc += (outputs.argmax(dim=1) == labels).float().sum().item()
            pbar.set_description(f'Train Iter: {epoch+1:03}/{num_epochs:03}  Loss: {loss:3.4f}')
            pbar.update()
        running_loss_avg = running_loss / len(dataset_train)
        running_acc_avg = running_acc / len(dataset_train)
        train_loss.append(running_loss_avg)
        train_acc.append(running_acc_avg)
        print(f"Epoch {epoch+1}, Loss: {running_loss_avg}, Accuracy: {running_acc_avg}")

        model.eval()
        with torch.no_grad():
            class_counts = [0] * 50
            running_loss = 0.0
            running_acc = 0.0
            pbar = tqdm(data_loader_val, colour='red', total=len(data_loader_val))
            for images, labels in pbar:
                images, labels = images.to(device), labels.to(device)
                outputs = model(images)
                predictions = outputs.argmax(dim=1)
                loss = criterion(outputs, labels)
                running_loss += loss.item()*len(labels)
                running_acc += (predictions == labels).float().sum().item()
                pbar.set_description(f'Valid Iter: {epoch+1:03}/{num_epochs:03}  Loss: {loss:3.4f}')
                pbar.update()

                for pred_label in predictions:
                    class_counts[pred_label] += 1

            running_loss_avg = running_loss / len(dataset_val)
            running_acc_avg = running_acc / len(dataset_val)
            val_loss.append(running_loss_avg)
            val_acc.append(running_acc_avg)
            print(f"Validation Loss: {running_loss_avg}, Accuracy: {running_acc_avg}")
            print(f"Class Counts: {class_counts}")
            if running_loss_avg < best_val_loss:
                best_val_loss = running_loss
                best_model = model
                torch.save(model.state_dict(), model_path)
        print("======================================")
    print("Finished Training")


    model.eval()
    with torch.no_grad():
        class_counts = [0] * 50
        running_loss = 0.0
        running_acc = 0.0
        predictions = []
        true_labels = []
        pbar = tqdm(data_loader_test, colour='red', total=len(data_loader_test))
        for images, labels in pbar:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)

            _, predicted = torch.max(outputs, 1)
            predictions.extend(predicted.cpu().numpy())
            true_labels.extend(labels.cpu().numpy())
            loss = criterion(outputs, labels)
            running_loss += loss.item()*len(labels)
            running_acc += (outputs.argmax(dim=1) == labels).float().sum().item()
            pbar.set_description(f'Test Iter: {epoch+1:03}/{num_epochs:03}  Loss: {loss:3.4f}')
            pbar.update()

        for pred_label in predictions:
            class_counts[pred_label] += 1
        
        precision = precision_score(true_labels, predictions, average='weighted')
        recall = recall_score(true_labels, predictions, average='weighted')
        f1 = f1_score(true_labels, predictions, average='weighted')
        print(f"Test Loss: {running_loss/len(dataset_test)}, Accuracy: {running_acc/len(dataset_test)}")
        print(f"Precision: {precision}, Recall: {recall}, F1: {f1}")
        print(f"Class Counts: {class_counts}")
    print("Finished Testing")

    train_record = {"train_loss": train_loss, "train_acc": train_acc, "val_loss": val_loss, "val_acc": val_acc}
    test_record = {"true_labels": true_labels, "predictions": predictions, "precision": precision, "recall": recall, "f1": f1, "class_counts": class_counts}


    with open(f'{save_path}/train_record/train_record_lr{lr}_bn_dropout50.pkl', 'wb') as f:
        pickle.dump(train_record, f)
    with open(f'{save_path}/test_record/test_record_lr{lr}_bn_dropout50.pkl', 'wb') as f:
        pickle.dump(test_record, f)


Train Iter: 001/010  Loss: 3.4875: 100%|██████████| 1979/1979 [04:22<00:00,  7.54it/s]


Epoch 1, Loss: 3.6021053779412293, Accuracy: 0.08938018160284247


Valid Iter: 001/010  Loss: 2.4024: 100%|██████████| 15/15 [00:00<00:00, 22.41it/s]


Validation Loss: 3.2657573223114014, Accuracy: 0.15777777777777777
Class Counts: [15, 12, 8, 39, 16, 2, 1, 2, 1, 4, 2, 2, 0, 0, 9, 5, 18, 0, 7, 15, 7, 10, 12, 34, 0, 1, 1, 2, 8, 2, 8, 0, 18, 17, 26, 2, 2, 6, 6, 7, 9, 16, 10, 2, 6, 10, 33, 6, 22, 9]


Train Iter: 002/010  Loss: 3.3236: 100%|██████████| 1979/1979 [04:23<00:00,  7.52it/s]


...

Epoch 10, Loss: 1.8503187420525384, Accuracy: 0.4885748124753257


Valid Iter: 010/010  Loss: 0.9684: 100%|██████████| 15/15 [00:00<00:00, 22.36it/s]


Validation Loss: 2.6346687671873306, Accuracy: 0.2822222222222222
Class Counts: [7, 13, 11, 10, 17, 6, 5, 14, 5, 6, 2, 6, 3, 9, 13, 11, 6, 14, 9, 14, 13, 4, 14, 8, 3, 10, 12, 3, 7, 12, 8, 0, 11, 12, 11, 9, 14, 6, 6, 11, 15, 4, 8, 10, 9, 8, 10, 9, 14, 8]
Finished Training


Test Iter: 010/010  Loss: 0.9684: 100%|██████████| 15/15 [00:00<00:00, 17.78it/s]

Test Loss: 0.9684146642684937, Accuracy: 0.2577777777777778
Precision: 0.28378178194354664, Recall: 0.2577777777777778, F1: 0.2538292060880296
Class Counts: [8, 10, 9, 12, 7, 11, 5, 12, 3, 4, 3, 6, 2, 15, 16, 17, 13, 15, 13, 8, 7, 2, 7, 13, 7, 15, 6, 6, 6, 8, 4, 6, 8, 26, 10, 10, 8, 6, 10, 10, 10, 7, 2, 8, 7, 11, 9, 11, 12, 9]
Finished Testing


In [5]:
if torch.cuda.is_available():
    device = torch.device("cuda")
else:
    device = torch.device("cpu")
class SelfAttention(nn.Module):
    def __init__(self, in_channels):
        super(SelfAttention, self).__init__()
        self.query_conv = nn.Conv2d(in_channels, in_channels // 8, kernel_size=1)
        self.key_conv = nn.Conv2d(in_channels, in_channels // 8, kernel_size=1)
        self.value_conv = nn.Conv2d(in_channels, in_channels, kernel_size=1)
        self.gamma = nn.Parameter(torch.zeros(1))
        
    def forward(self, x):
        batch_size, C, width, height = x.size()
        
        # 计算查询、键和值
        proj_query = self.query_conv(x).view(batch_size, -1, width * height).permute(0, 2, 1)
        proj_key = self.key_conv(x).view(batch_size, -1, width * height)
        energy = torch.bmm(proj_query, proj_key)
        
        # 计算注意力权重并应用到值上
        attention = F.softmax(energy, dim=-1)
        proj_value = self.value_conv(x).view(batch_size, -1, width * height)
        out = torch.bmm(proj_value, attention.permute(0, 2, 1))
        
        # 调整输出并加上残差连接
        out = out.view(batch_size, C, width, height)
        out = self.gamma * out + x
        return out

class CNNModel(nn.Module):
    def __init__(self, num_classes=50):
        super(CNNModel, self).__init__()
        self.conv1 = nn.Conv2d(3, 64, kernel_size=3, padding=1)
        self.bn1 = nn.BatchNorm2d(64)
        self.conv2 = nn.Conv2d(64, 128, kernel_size=3, padding=1)
        self.bn2 = nn.BatchNorm2d(128)
        self.conv3 = nn.Conv2d(128, 256, kernel_size=3, padding=1)
        self.bn3 = nn.BatchNorm2d(256)
        self.attention1 = SelfAttention(256)

        self.fc1 = nn.Linear(256*28*28, 1024)
        self.fc2 = nn.Linear(1024, num_classes)
        self.activation = nn.ReLU()
        self.flatten = nn.Flatten()
        self.pool = nn.MaxPool2d(2, 2)
        self.dropout25 = nn.Dropout(0.25)
        self.dropout50 = nn.Dropout(0.5)
        
    def forward(self, x):
        x = self.pool(self.activation(self.bn1(self.conv1(x))))
        x = self.pool(self.activation(self.bn2(self.conv2(x))))
        x = self.pool(self.activation(self.bn3(self.conv3(x))))
        x = self.attention1(x)
        x = self.dropout25(self.flatten(x))
        x = self.dropout50(self.activation(self.fc1(x)))
        x = self.fc2(x)
        return x


for lr in [1e-5]:
    model = CNNModel(num_classes=50).to(device)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=lr)
    num_epochs = 10
    save_path = 'record/Q2_less_layers'
    rec_name = '6layers'
    model_path = f'{save_path}/model_weight/model_weight_lr{lr}_{rec_name}.pth'
    best_val_loss = float('inf')
    best_model = None
    train_loss = []
    train_acc = []
    val_loss = []
    val_acc = []
    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0
        running_acc = 0.0
        pbar = tqdm(data_loader_train, colour='green', total=len(data_loader_train))
        for images, labels in pbar:
            images, labels = images.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()*len(labels)
            running_acc += (outputs.argmax(dim=1) == labels).float().sum().item()
            pbar.set_description(f'Train Iter: {epoch+1:03}/{num_epochs:03}  Loss: {loss:3.4f}')
            pbar.update()
        running_loss_avg = running_loss / len(dataset_train)
        running_acc_avg = running_acc / len(dataset_train)
        train_loss.append(running_loss_avg)
        train_acc.append(running_acc_avg)
        print(f"Epoch {epoch+1}, Loss: {running_loss_avg}, Accuracy: {running_acc_avg}")

        model.eval()
        with torch.no_grad():
            class_counts = [0] * 50
            running_loss = 0.0
            running_acc = 0.0
            pbar = tqdm(data_loader_val, colour='red', total=len(data_loader_val))
            for images, labels in pbar:
                images, labels = images.to(device), labels.to(device)
                outputs = model(images)
                predictions = outputs.argmax(dim=1)
                loss = criterion(outputs, labels)
                running_loss += loss.item()*len(labels)
                running_acc += (predictions == labels).float().sum().item()
                pbar.set_description(f'Valid Iter: {epoch+1:03}/{num_epochs:03}  Loss: {loss:3.4f}')
                pbar.update()

                for pred_label in predictions:
                    class_counts[pred_label] += 1

            running_loss_avg = running_loss / len(dataset_val)
            running_acc_avg = running_acc / len(dataset_val)
            val_loss.append(running_loss_avg)
            val_acc.append(running_acc_avg)
            print(f"Validation Loss: {running_loss_avg}, Accuracy: {running_acc_avg}")
            print(f"Class Counts: {class_counts}")
            if running_loss_avg < best_val_loss:
                best_val_loss = running_loss
                best_model = model
                torch.save(model.state_dict(), model_path)
        print("======================================")
    print("Finished Training")


    model.eval()
    with torch.no_grad():
        class_counts = [0] * 50
        running_loss = 0.0
        running_acc = 0.0
        predictions = []
        true_labels = []
        pbar = tqdm(data_loader_test, colour='red', total=len(data_loader_test))
        for images, labels in pbar:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)

            _, predicted = torch.max(outputs, 1)
            predictions.extend(predicted.cpu().numpy())
            true_labels.extend(labels.cpu().numpy())
            loss = criterion(outputs, labels)
            running_loss += loss.item()*len(labels)
            running_acc += (outputs.argmax(dim=1) == labels).float().sum().item()
            pbar.set_description(f'Test Iter: {epoch+1:03}/{num_epochs:03}  Loss: {loss:3.4f}')
            pbar.update()

        for pred_label in predictions:
            class_counts[pred_label] += 1
        
        precision = precision_score(true_labels, predictions, average='weighted')
        recall = recall_score(true_labels, predictions, average='weighted')
        f1 = f1_score(true_labels, predictions, average='weighted')
        print(f"Test Loss: {running_loss/len(dataset_test)}, Accuracy: {running_acc/len(dataset_test)}")
        print(f"Precision: {precision}, Recall: {recall}, F1: {f1}")
        print(f"Class Counts: {class_counts}")
    print("Finished Testing")

    train_record = {"train_loss": train_loss, "train_acc": train_acc, "val_loss": val_loss, "val_acc": val_acc}
    test_record = {"true_labels": true_labels, "predictions": predictions, "precision": precision, "recall": recall, "f1": f1, "class_counts": class_counts}


    with open(f'{save_path}/train_record/train_record_lr{lr}_{rec_name}.pkl', 'wb') as f:
        pickle.dump(train_record, f)
    with open(f'{save_path}/test_record/test_record_lr{lr}_{rec_name}.pkl', 'wb') as f:
        pickle.dump(test_record, f)


Train Iter: 001/010  Loss: 3.4272: 100%|██████████| 1979/1979 [02:52<00:00, 11.50it/s]


Epoch 1, Loss: 3.4936507674125736, Accuracy: 0.10836162652980655


Valid Iter: 001/010  Loss: 1.8520: 100%|██████████| 15/15 [00:00<00:00, 25.47it/s]


Validation Loss: 3.0952224588394164, Accuracy: 0.19555555555555557
Class Counts: [18, 12, 9, 14, 25, 4, 1, 4, 2, 0, 0, 2, 0, 1, 1, 27, 8, 0, 2, 3, 6, 5, 4, 30, 8, 0, 0, 16, 1, 22, 20, 1, 17, 15, 13, 4, 7, 11, 13, 10, 3, 11, 3, 22, 6, 10, 20, 1, 18, 20]


Train Iter: 002/010  Loss: 2.9536: 100%|██████████| 1979/1979 [02:52<00:00, 11.46it/s]


...

Epoch 10, Loss: 1.340624063271004, Accuracy: 0.6348835373075404


Valid Iter: 010/010  Loss: 0.1901: 100%|██████████| 15/15 [00:00<00:00, 25.98it/s]


Validation Loss: 2.3425543109575906, Accuracy: 0.3377777777777778
Class Counts: [8, 9, 16, 8, 21, 8, 2, 5, 7, 3, 8, 3, 5, 7, 10, 9, 14, 10, 8, 9, 8, 13, 15, 9, 3, 9, 8, 7, 7, 9, 4, 10, 8, 7, 10, 10, 12, 10, 11, 8, 13, 9, 10, 17, 13, 10, 9, 5, 11, 5]
Finished Training


Test Iter: 010/010  Loss: 0.1901: 100%|██████████| 15/15 [00:00<00:00, 25.92it/s]

Test Loss: 0.19008785486221313, Accuracy: 0.37333333333333335
Precision: 0.37970499435205324, Recall: 0.37333333333333335, F1: 0.3620732739568653
Class Counts: [12, 10, 12, 8, 8, 8, 8, 7, 5, 6, 4, 7, 5, 11, 11, 11, 16, 9, 9, 13, 3, 13, 17, 10, 4, 11, 8, 9, 7, 10, 5, 9, 5, 10, 8, 7, 8, 8, 8, 6, 16, 11, 9, 9, 15, 12, 12, 9, 8, 3]
Finished Testing


In [4]:
if torch.cuda.is_available():
    device = torch.device("cuda")
else:
    device = torch.device("cpu")
class SelfAttention(nn.Module):
    def __init__(self, in_channels):
        super(SelfAttention, self).__init__()
        self.query_conv = nn.Conv2d(in_channels, in_channels // 8, kernel_size=1)
        self.key_conv = nn.Conv2d(in_channels, in_channels // 8, kernel_size=1)
        self.value_conv = nn.Conv2d(in_channels, in_channels, kernel_size=1)
        self.gamma = nn.Parameter(torch.zeros(1))
        
    def forward(self, x):
        batch_size, C, width, height = x.size()
        
        # 计算查询、键和值
        proj_query = self.query_conv(x).view(batch_size, -1, width * height).permute(0, 2, 1)
        proj_key = self.key_conv(x).view(batch_size, -1, width * height)
        energy = torch.bmm(proj_query, proj_key)
        
        # 计算注意力权重并应用到值上
        attention = F.softmax(energy, dim=-1)
        proj_value = self.value_conv(x).view(batch_size, -1, width * height)
        out = torch.bmm(proj_value, attention.permute(0, 2, 1))
        
        # 调整输出并加上残差连接
        out = out.view(batch_size, C, width, height)
        out = self.gamma * out + x
        return out

class CNNModel(nn.Module):
    def __init__(self, num_classes=50):
        super(CNNModel, self).__init__()
        self.conv1 = nn.Conv2d(3, 64, kernel_size=3, padding=1)
        self.bn1 = nn.BatchNorm2d(64)
        self.conv2 = nn.Conv2d(64, 128, kernel_size=3, padding=1)
        self.bn2 = nn.BatchNorm2d(128)
        self.conv3 = nn.Conv2d(128, 256, kernel_size=3, padding=1)
        self.bn3 = nn.BatchNorm2d(256)
        self.attention1 = SelfAttention(256)

        self.fc1 = nn.Linear(256*28*28, 2048)
        self.fc2 = nn.Linear(2048, num_classes)
        self.activation = nn.ReLU()
        self.flatten = nn.Flatten()
        self.pool = nn.MaxPool2d(2, 2)
        self.dropout25 = nn.Dropout(0.25)
        self.dropout50 = nn.Dropout(0.5)
        
    def forward(self, x):
        x = self.pool(self.activation(self.bn1(self.conv1(x))))
        x = self.pool(self.activation(self.bn2(self.conv2(x))))
        x = self.pool(self.activation(self.bn3(self.conv3(x))))
        x = self.attention1(x)
        x = self.dropout25(self.flatten(x))
        x = self.dropout50(self.activation(self.fc1(x)))
        x = self.fc2(x)
        return x


for lr in [1e-5]:
    model = CNNModel(num_classes=50).to(device)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=lr)
    num_epochs = 10
    save_path = 'record/Q2_less_layers'
    rec_name = '6layers_2048'
    model_path = f'{save_path}/model_weight/model_weight_lr{lr}_{rec_name}.pth'
    best_val_loss = float('inf')
    best_model = None
    train_loss = []
    train_acc = []
    val_loss = []
    val_acc = []
    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0
        running_acc = 0.0
        pbar = tqdm(data_loader_train, colour='green', total=len(data_loader_train))
        for images, labels in pbar:
            images, labels = images.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()*len(labels)
            running_acc += (outputs.argmax(dim=1) == labels).float().sum().item()
            pbar.set_description(f'Train Iter: {epoch+1:03}/{num_epochs:03}  Loss: {loss:3.4f}')
            pbar.update()
        running_loss_avg = running_loss / len(dataset_train)
        running_acc_avg = running_acc / len(dataset_train)
        train_loss.append(running_loss_avg)
        train_acc.append(running_acc_avg)
        print(f"Epoch {epoch+1}, Loss: {running_loss_avg}, Accuracy: {running_acc_avg}")

        model.eval()
        with torch.no_grad():
            class_counts = [0] * 50
            running_loss = 0.0
            running_acc = 0.0
            pbar = tqdm(data_loader_val, colour='red', total=len(data_loader_val))
            for images, labels in pbar:
                images, labels = images.to(device), labels.to(device)
                outputs = model(images)
                predictions = outputs.argmax(dim=1)
                loss = criterion(outputs, labels)
                running_loss += loss.item()*len(labels)
                running_acc += (predictions == labels).float().sum().item()
                pbar.set_description(f'Valid Iter: {epoch+1:03}/{num_epochs:03}  Loss: {loss:3.4f}')
                pbar.update()

                for pred_label in predictions:
                    class_counts[pred_label] += 1

            running_loss_avg = running_loss / len(dataset_val)
            running_acc_avg = running_acc / len(dataset_val)
            val_loss.append(running_loss_avg)
            val_acc.append(running_acc_avg)
            print(f"Validation Loss: {running_loss_avg}, Accuracy: {running_acc_avg}")
            print(f"Class Counts: {class_counts}")
            if running_loss_avg < best_val_loss:
                best_val_loss = running_loss
                best_model = model
                torch.save(model.state_dict(), model_path)
        print("======================================")
    print("Finished Training")


    model.eval()
    with torch.no_grad():
        class_counts = [0] * 50
        running_loss = 0.0
        running_acc = 0.0
        predictions = []
        true_labels = []
        pbar = tqdm(data_loader_test, colour='red', total=len(data_loader_test))
        for images, labels in pbar:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)

            _, predicted = torch.max(outputs, 1)
            predictions.extend(predicted.cpu().numpy())
            true_labels.extend(labels.cpu().numpy())
            loss = criterion(outputs, labels)

            running_loss += loss.item()*len(labels)
            running_acc += (outputs.argmax(dim=1) == labels).float().sum().item()
            pbar.set_description(f'Test Iter: {epoch+1:03}/{num_epochs:03}  Loss: {loss:3.4f}')
            pbar.update()

        for pred_label in predictions:
            class_counts[pred_label] += 1
        
        precision = precision_score(true_labels, predictions, average='weighted')
        recall = recall_score(true_labels, predictions, average='weighted')
        f1 = f1_score(true_labels, predictions, average='weighted')
        print(f"Test Loss: {running_loss/len(dataset_test)}, Accuracy: {running_acc/len(dataset_test)}")
        print(f"Precision: {precision}, Recall: {recall}, F1: {f1}")
        print(f"Class Counts: {class_counts}")
    print("Finished Testing")

    train_record = {"train_loss": train_loss, "train_acc": train_acc, "val_loss": val_loss, "val_acc": val_acc}
    test_record = {"true_labels": true_labels, "predictions": predictions, "precision": precision, "recall": recall, "f1": f1, "class_counts": class_counts}


    with open(f'{save_path}/train_record/train_record_lr{lr}_{rec_name}.pkl', 'wb') as f:
        pickle.dump(train_record, f)
    with open(f'{save_path}/test_record/test_record_lr{lr}_{rec_name}.pkl', 'wb') as f:
        pickle.dump(test_record, f)


Train Iter: 001/010  Loss: 3.0179: 100%|██████████| 1979/1979 [03:43<00:00,  8.84it/s]


Epoch 1, Loss: 3.3699071711680655, Accuracy: 0.12876431109356495


Valid Iter: 001/010  Loss: 1.2763: 100%|██████████| 15/15 [00:00<00:00, 24.32it/s]


Validation Loss: 2.9545362122853596, Accuracy: 0.21555555555555556
Class Counts: [5, 10, 14, 7, 28, 12, 8, 0, 5, 2, 2, 0, 1, 1, 20, 3, 0, 6, 4, 11, 15, 11, 6, 12, 4, 1, 0, 14, 0, 10, 7, 0, 28, 8, 6, 16, 5, 12, 19, 3, 6, 27, 10, 18, 7, 9, 13, 6, 15, 23]


Train Iter: 002/010  Loss: 2.7731: 100%|██████████| 1979/1979 [03:42<00:00,  8.88it/s]


...

Epoch 10, Loss: 0.7323346991168149, Accuracy: 0.8213028030003948


Valid Iter: 010/010  Loss: 0.0581: 100%|██████████| 15/15 [00:00<00:00, 24.80it/s]


Validation Loss: 2.386636883947584, Accuracy: 0.3466666666666667
Class Counts: [4, 12, 12, 11, 18, 9, 6, 4, 6, 10, 6, 4, 4, 8, 5, 11, 11, 22, 11, 7, 14, 12, 7, 7, 8, 10, 5, 11, 7, 10, 5, 5, 7, 5, 8, 9, 8, 11, 5, 8, 11, 13, 9, 8, 5, 9, 12, 10, 11, 19]
Finished Training


Test Iter: 010/010  Loss: 0.0581: 100%|██████████| 15/15 [00:00<00:00, 24.77it/s]

Test Loss: 0.058064818382263184, Accuracy: 0.3511111111111111
Precision: 0.36408938773644656, Recall: 0.3511111111111111, F1: 0.34448776757808924
Class Counts: [8, 8, 13, 9, 12, 7, 7, 5, 5, 10, 3, 12, 3, 9, 6, 9, 15, 17, 12, 7, 7, 6, 8, 11, 8, 11, 4, 8, 7, 11, 9, 12, 3, 8, 7, 12, 8, 11, 10, 7, 8, 12, 12, 9, 7, 11, 18, 6, 8, 14]
Finished Testing


In [6]:
if torch.cuda.is_available():
    device = torch.device("cuda")
else:
    device = torch.device("cpu")
class SelfAttention(nn.Module):
    def __init__(self, in_channels):
        super(SelfAttention, self).__init__()
        self.query_conv = nn.Conv2d(in_channels, in_channels // 8, kernel_size=1)
        self.key_conv = nn.Conv2d(in_channels, in_channels // 8, kernel_size=1)
        self.value_conv = nn.Conv2d(in_channels, in_channels, kernel_size=1)
        self.gamma = nn.Parameter(torch.zeros(1))
        
    def forward(self, x):
        batch_size, C, width, height = x.size()
        
        # 计算查询、键和值
        proj_query = self.query_conv(x).view(batch_size, -1, width * height).permute(0, 2, 1)
        proj_key = self.key_conv(x).view(batch_size, -1, width * height)
        energy = torch.bmm(proj_query, proj_key)
        
        # 计算注意力权重并应用到值上
        attention = F.softmax(energy, dim=-1)
        proj_value = self.value_conv(x).view(batch_size, -1, width * height)
        out = torch.bmm(proj_value, attention.permute(0, 2, 1))
        
        # 调整输出并加上残差连接
        out = out.view(batch_size, C, width, height)
        out = self.gamma * out + x
        return out

class CNNModel(nn.Module):
    def __init__(self, num_classes=50):
        super(CNNModel, self).__init__()
        self.conv1 = nn.Conv2d(3, 64, kernel_size=3, padding=1)
        self.bn1 = nn.BatchNorm2d(64)
        self.conv2 = nn.Conv2d(64, 128, kernel_size=3, padding=1)
        self.bn2 = nn.BatchNorm2d(128)
        self.conv3 = nn.Conv2d(128, 256, kernel_size=3, padding=1)
        self.bn3 = nn.BatchNorm2d(256)
        self.attention1 = SelfAttention(256)

        self.fc1 = nn.Linear(256*28*28, 1024)
        self.fc2 = nn.Linear(1024, num_classes)
        self.activation = nn.ReLU()
        self.flatten = nn.Flatten()
        self.pool = nn.MaxPool2d(2, 2)
        # self.dropout25 = nn.Dropout(0.25)
        self.dropout50 = nn.Dropout(0.5)
        
    def forward(self, x):
        x = self.pool(self.activation(self.bn1(self.conv1(x))))
        x = self.pool(self.activation(self.bn2(self.conv2(x))))
        x = self.pool(self.activation(self.bn3(self.conv3(x))))
        x = self.attention1(x)
        x = self.dropout50(self.flatten(x))
        x = self.dropout50(self.activation(self.fc1(x)))
        x = self.fc2(x)
        return x


for lr in [1e-5]:
    model = CNNModel(num_classes=50).to(device)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=lr)
    num_epochs = 10
    save_path = 'record/Q2_less_layers'
    rec_name = '6layers_dropout50'
    model_path = f'{save_path}/model_weight/model_weight_lr{lr}_{rec_name}.pth'
    best_val_loss = float('inf')
    best_model = None
    train_loss = []
    train_acc = []
    val_loss = []
    val_acc = []
    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0
        running_acc = 0.0
        pbar = tqdm(data_loader_train, colour='green', total=len(data_loader_train))
        for images, labels in pbar:
            images, labels = images.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()*len(labels)
            running_acc += (outputs.argmax(dim=1) == labels).float().sum().item()
            pbar.set_description(f'Train Iter: {epoch+1:03}/{num_epochs:03}  Loss: {loss:3.4f}')
            pbar.update()
        running_loss_avg = running_loss / len(dataset_train)
        running_acc_avg = running_acc / len(dataset_train)
        train_loss.append(running_loss_avg)
        train_acc.append(running_acc_avg)
        print(f"Epoch {epoch+1}, Loss: {running_loss_avg}, Accuracy: {running_acc_avg}")

        model.eval()
        with torch.no_grad():
            class_counts = [0] * 50
            running_loss = 0.0
            running_acc = 0.0
            pbar = tqdm(data_loader_val, colour='red', total=len(data_loader_val))
            for images, labels in pbar:
                images, labels = images.to(device), labels.to(device)
                outputs = model(images)
                predictions = outputs.argmax(dim=1)
                loss = criterion(outputs, labels)
                running_loss += loss.item()*len(labels)
                running_acc += (predictions == labels).float().sum().item()
                pbar.set_description(f'Valid Iter: {epoch+1:03}/{num_epochs:03}  Loss: {loss:3.4f}')
                pbar.update()

                for pred_label in predictions:
                    class_counts[pred_label] += 1

            running_loss_avg = running_loss / len(dataset_val)
            running_acc_avg = running_acc / len(dataset_val)
            val_loss.append(running_loss_avg)
            val_acc.append(running_acc_avg)
            print(f"Validation Loss: {running_loss_avg}, Accuracy: {running_acc_avg}")
            print(f"Class Counts: {class_counts}")
            if running_loss_avg < best_val_loss:
                best_val_loss = running_loss
                best_model = model
                torch.save(model.state_dict(), model_path)
        print("======================================")
    print("Finished Training")


    model.eval()
    with torch.no_grad():
        class_counts = [0] * 50
        running_loss = 0.0
        running_acc = 0.0
        predictions = []
        true_labels = []
        pbar = tqdm(data_loader_test, colour='red', total=len(data_loader_test))
        for images, labels in pbar:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)

            _, predicted = torch.max(outputs, 1)
            predictions.extend(predicted.cpu().numpy())
            true_labels.extend(labels.cpu().numpy())
            loss = criterion(outputs, labels)

            running_loss += loss.item()*len(labels)
            running_acc += (outputs.argmax(dim=1) == labels).float().sum().item()
            pbar.set_description(f'Test Iter: {epoch+1:03}/{num_epochs:03}  Loss: {loss:3.4f}')
            pbar.update()

        for pred_label in predictions:
            class_counts[pred_label] += 1
        
        precision = precision_score(true_labels, predictions, average='weighted')
        recall = recall_score(true_labels, predictions, average='weighted')
        f1 = f1_score(true_labels, predictions, average='weighted')
        print(f"Test Loss: {running_loss/len(dataset_test)}, Accuracy: {running_acc/len(dataset_test)}")
        print(f"Precision: {precision}, Recall: {recall}, F1: {f1}")
        print(f"Class Counts: {class_counts}")
    print("Finished Testing")

    train_record = {"train_loss": train_loss, "train_acc": train_acc, "val_loss": val_loss, "val_acc": val_acc}
    test_record = {"true_labels": true_labels, "predictions": predictions, "precision": precision, "recall": recall, "f1": f1, "class_counts": class_counts}


    with open(f'{save_path}/train_record/train_record_lr{lr}_{rec_name}.pkl', 'wb') as f:
        pickle.dump(train_record, f)
    with open(f'{save_path}/test_record/test_record_lr{lr}_{rec_name}.pkl', 'wb') as f:
        pickle.dump(test_record, f)


Train Iter: 001/010  Loss: 3.3324: 100%|██████████| 1979/1979 [02:51<00:00, 11.55it/s]


Epoch 1, Loss: 3.472888354923807, Accuracy: 0.11063560994867745


Valid Iter: 001/010  Loss: 2.1215: 100%|██████████| 15/15 [00:00<00:00, 25.24it/s]


Validation Loss: 3.070883650249905, Accuracy: 0.19333333333333333
Class Counts: [10, 1, 8, 43, 12, 4, 3, 1, 9, 3, 1, 0, 0, 1, 21, 5, 7, 4, 6, 1, 2, 10, 2, 17, 17, 5, 1, 34, 3, 1, 1, 2, 12, 9, 14, 7, 4, 17, 2, 16, 5, 9, 37, 8, 10, 15, 11, 13, 11, 15]


Train Iter: 002/010  Loss: 3.0650: 100%|██████████| 1979/1979 [02:52<00:00, 11.48it/s]


...

Epoch 10, Loss: 1.2142170782123118, Accuracy: 0.6728464271614686


Valid Iter: 010/010  Loss: 0.2201: 100%|██████████| 15/15 [00:00<00:00, 25.49it/s]


Validation Loss: 2.34850836303499, Accuracy: 0.3333333333333333
Class Counts: [9, 11, 10, 16, 15, 11, 6, 7, 12, 5, 4, 4, 5, 13, 5, 4, 6, 11, 9, 11, 12, 5, 6, 9, 10, 13, 4, 13, 5, 9, 3, 6, 13, 10, 8, 9, 11, 7, 6, 13, 13, 5, 6, 18, 7, 14, 11, 7, 12, 11]
Finished Training


Test Iter: 010/010  Loss: 0.2201: 100%|██████████| 15/15 [00:00<00:00, 23.73it/s]


Test Loss: 0.22008007764816284, Accuracy: 0.37333333333333335
Precision: 0.3627560495060495, Recall: 0.37333333333333335, F1: 0.3582361576444225
Class Counts: [12, 9, 11, 11, 6, 11, 8, 7, 11, 3, 5, 2, 5, 15, 6, 9, 6, 4, 12, 9, 9, 10, 13, 13, 3, 9, 7, 9, 9, 11, 5, 4, 8, 16, 8, 11, 8, 8, 9, 8, 10, 11, 13, 10, 10, 15, 14, 8, 10, 9]
Finished Testing


In [8]:
if torch.cuda.is_available():
    device = torch.device("cuda")
else:
    device = torch.device("cpu")
class SelfAttention(nn.Module):
    def __init__(self, in_channels):
        super(SelfAttention, self).__init__()
        self.query_conv = nn.Conv2d(in_channels, in_channels // 8, kernel_size=1)
        self.key_conv = nn.Conv2d(in_channels, in_channels // 8, kernel_size=1)
        self.value_conv = nn.Conv2d(in_channels, in_channels, kernel_size=1)
        self.gamma = nn.Parameter(torch.zeros(1))
        
    def forward(self, x):
        batch_size, C, width, height = x.size()
        
        # 计算查询、键和值
        proj_query = self.query_conv(x).view(batch_size, -1, width * height).permute(0, 2, 1)
        proj_key = self.key_conv(x).view(batch_size, -1, width * height)
        energy = torch.bmm(proj_query, proj_key)
        
        # 计算注意力权重并应用到值上
        attention = F.softmax(energy, dim=-1)
        proj_value = self.value_conv(x).view(batch_size, -1, width * height)
        out = torch.bmm(proj_value, attention.permute(0, 2, 1))
        
        # 调整输出并加上残差连接
        out = out.view(batch_size, C, width, height)
        out = self.gamma * out + x
        return out

class CNNModel(nn.Module):
    def __init__(self, num_classes=50):
        super(CNNModel, self).__init__()
        self.conv1 = nn.Conv2d(3, 64, kernel_size=3, padding=1)
        self.bn1 = nn.BatchNorm2d(64)
        self.conv2 = nn.Conv2d(64, 128, kernel_size=3, padding=1)
        self.bn2 = nn.BatchNorm2d(128)
        self.conv3 = nn.Conv2d(128, 256, kernel_size=3, padding=1)
        self.bn3 = nn.BatchNorm2d(256)
        self.attention1 = SelfAttention(256)

        self.fc1 = nn.Linear(256*28*28, 2048)
        self.fc2 = nn.Linear(2048, num_classes)
        self.activation = nn.ReLU()
        self.flatten = nn.Flatten()
        self.pool = nn.MaxPool2d(2, 2)
        # self.dropout25 = nn.Dropout(0.25)
        self.dropout50 = nn.Dropout(0.5)
        
    def forward(self, x):
        x = self.pool(self.activation(self.bn1(self.conv1(x))))
        x = self.pool(self.activation(self.bn2(self.conv2(x))))
        x = self.pool(self.activation(self.bn3(self.conv3(x))))
        x = self.attention1(x)
        x = self.dropout50(self.flatten(x))
        x = self.dropout50(self.activation(self.fc1(x)))
        x = self.fc2(x)
        return x


for lr in [1e-5]:
    model = CNNModel(num_classes=50).to(device)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=lr, weight_decay=1e-4)
    num_epochs = 10
    save_path = 'record/Q2_less_layers'
    rec_name = '6layers_2048_dropout50_weightdecay'
    model_path = f'{save_path}/model_weight/model_weight_lr{lr}_{rec_name}.pth'
    best_val_loss = float('inf')
    best_model = None
    train_loss = []
    train_acc = []
    val_loss = []
    val_acc = []
    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0
        running_acc = 0.0
        pbar = tqdm(data_loader_train, colour='green', total=len(data_loader_train))
        for images, labels in pbar:
            images, labels = images.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()*len(labels)
            running_acc += (outputs.argmax(dim=1) == labels).float().sum().item()
            pbar.set_description(f'Train Iter: {epoch+1:03}/{num_epochs:03}  Loss: {loss:3.4f}')
            pbar.update()
        running_loss_avg = running_loss / len(dataset_train)
        running_acc_avg = running_acc / len(dataset_train)
        train_loss.append(running_loss_avg)
        train_acc.append(running_acc_avg)
        print(f"Epoch {epoch+1}, Loss: {running_loss_avg}, Accuracy: {running_acc_avg}")

        model.eval()
        with torch.no_grad():
            class_counts = [0] * 50
            running_loss = 0.0
            running_acc = 0.0
            pbar = tqdm(data_loader_val, colour='red', total=len(data_loader_val))
            for images, labels in pbar:
                images, labels = images.to(device), labels.to(device)
                outputs = model(images)
                predictions = outputs.argmax(dim=1)
                loss = criterion(outputs, labels)
                running_loss += loss.item()*len(labels)
                running_acc += (predictions == labels).float().sum().item()
                pbar.set_description(f'Valid Iter: {epoch+1:03}/{num_epochs:03}  Loss: {loss:3.4f}')
                pbar.update()

                for pred_label in predictions:
                    class_counts[pred_label] += 1

            running_loss_avg = running_loss / len(dataset_val)
            running_acc_avg = running_acc / len(dataset_val)
            val_loss.append(running_loss_avg)
            val_acc.append(running_acc_avg)
            print(f"Validation Loss: {running_loss_avg}, Accuracy: {running_acc_avg}")
            print(f"Class Counts: {class_counts}")
            if running_loss_avg < best_val_loss:
                best_val_loss = running_loss
                best_model = model
                torch.save(model.state_dict(), model_path)
        print("======================================")
    print("Finished Training")


    model.eval()
    with torch.no_grad():
        class_counts = [0] * 50
        running_loss = 0.0
        running_acc = 0.0
        predictions = []
        true_labels = []
        pbar = tqdm(data_loader_test, colour='red', total=len(data_loader_test))
        for images, labels in pbar:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)

            _, predicted = torch.max(outputs, 1)
            predictions.extend(predicted.cpu().numpy())
            true_labels.extend(labels.cpu().numpy())
            loss = criterion(outputs, labels)

            running_loss += loss.item()*len(labels)
            running_acc += (outputs.argmax(dim=1) == labels).float().sum().item()
            pbar.set_description(f'Test Iter: {epoch+1:03}/{num_epochs:03}  Loss: {loss:3.4f}')
            pbar.update()

        for pred_label in predictions:
            class_counts[pred_label] += 1
        
        precision = precision_score(true_labels, predictions, average='weighted')
        recall = recall_score(true_labels, predictions, average='weighted')
        f1 = f1_score(true_labels, predictions, average='weighted')
        print(f"Test Loss: {running_loss/len(dataset_test)}, Accuracy: {running_acc/len(dataset_test)}")
        print(f"Precision: {precision}, Recall: {recall}, F1: {f1}")
        print(f"Class Counts: {class_counts}")
    print("Finished Testing")

    train_record = {"train_loss": train_loss, "train_acc": train_acc, "val_loss": val_loss, "val_acc": val_acc}
    test_record = {"true_labels": true_labels, "predictions": predictions, "precision": precision, "recall": recall, "f1": f1, "class_counts": class_counts}


    with open(f'{save_path}/train_record/train_record_lr{lr}_{rec_name}.pkl', 'wb') as f:
        pickle.dump(train_record, f)
    with open(f'{save_path}/test_record/test_record_lr{lr}_{rec_name}.pkl', 'wb') as f:
        pickle.dump(test_record, f)


Train Iter: 001/010  Loss: 3.3190: 100%|██████████| 1979/1979 [03:49<00:00,  8.64it/s]


Epoch 1, Loss: 3.3806793784455165, Accuracy: 0.12958547177260166


Valid Iter: 001/010  Loss: 2.2356: 100%|██████████| 15/15 [00:00<00:00, 25.04it/s]


Validation Loss: 2.991858303281996, Accuracy: 0.18222222222222223
Class Counts: [22, 35, 5, 0, 28, 11, 3, 3, 0, 5, 0, 0, 1, 0, 6, 19, 0, 1, 4, 15, 17, 9, 26, 6, 10, 2, 4, 21, 4, 8, 2, 9, 8, 16, 12, 6, 4, 9, 10, 8, 8, 5, 18, 2, 13, 17, 19, 6, 10, 3]


Train Iter: 002/010  Loss: 2.9143: 100%|██████████| 1979/1979 [03:49<00:00,  8.63it/s]


...

Epoch 10, Loss: 0.6632199572396759, Accuracy: 0.8438531385708646


Valid Iter: 010/010  Loss: 0.5004: 100%|██████████| 15/15 [00:00<00:00, 24.19it/s]


Validation Loss: 2.346022649606069, Accuracy: 0.3711111111111111
Class Counts: [11, 7, 10, 10, 17, 7, 3, 8, 9, 10, 4, 5, 4, 8, 9, 12, 5, 12, 8, 14, 14, 10, 16, 9, 7, 13, 6, 12, 8, 6, 3, 7, 9, 9, 7, 8, 6, 12, 12, 10, 11, 7, 7, 16, 12, 6, 6, 6, 12, 10]
Finished Training


Test Iter: 010/010  Loss: 0.5004: 100%|██████████| 15/15 [00:00<00:00, 23.50it/s]

Test Loss: 0.5004361271858215, Accuracy: 0.3622222222222222
Precision: 0.36843709231944527, Recall: 0.3622222222222222, F1: 0.3480189904545134
Class Counts: [9, 9, 9, 1, 13, 7, 7, 14, 12, 9, 4, 5, 6, 8, 10, 9, 12, 9, 11, 17, 8, 6, 12, 11, 4, 14, 2, 6, 5, 9, 7, 8, 4, 13, 10, 10, 7, 11, 10, 8, 10, 11, 11, 12, 10, 12, 12, 9, 11, 6]
Finished Testing


In [9]:
if torch.cuda.is_available():
    device = torch.device("cuda")
else:
    device = torch.device("cpu")
class SelfAttention(nn.Module):
    def __init__(self, in_channels):
        super(SelfAttention, self).__init__()
        self.query_conv = nn.Conv2d(in_channels, in_channels // 8, kernel_size=1)
        self.key_conv = nn.Conv2d(in_channels, in_channels // 8, kernel_size=1)
        self.value_conv = nn.Conv2d(in_channels, in_channels, kernel_size=1)
        self.gamma = nn.Parameter(torch.zeros(1))
        
    def forward(self, x):
        batch_size, C, width, height = x.size()
        
        # 计算查询、键和值
        proj_query = self.query_conv(x).view(batch_size, -1, width * height).permute(0, 2, 1)
        proj_key = self.key_conv(x).view(batch_size, -1, width * height)
        energy = torch.bmm(proj_query, proj_key)
        
        # 计算注意力权重并应用到值上
        attention = F.softmax(energy, dim=-1)
        proj_value = self.value_conv(x).view(batch_size, -1, width * height)
        out = torch.bmm(proj_value, attention.permute(0, 2, 1))
        
        # 调整输出并加上残差连接
        out = out.view(batch_size, C, width, height)
        out = self.gamma * out + x
        return out

class CNNModel(nn.Module):
    def __init__(self, num_classes=50):
        super(CNNModel, self).__init__()
        self.conv1 = nn.Conv2d(3, 64, kernel_size=3, padding=1)
        self.bn1 = nn.BatchNorm2d(64)
        self.conv2 = nn.Conv2d(64, 128, kernel_size=3, padding=1)
        self.bn2 = nn.BatchNorm2d(128)
        self.conv3 = nn.Conv2d(128, 256, kernel_size=3, padding=1)
        self.bn3 = nn.BatchNorm2d(256)
        self.attention1 = SelfAttention(256)

        self.fc1 = nn.Linear(256*28*28, 2048)
        self.fc2 = nn.Linear(2048, num_classes)
        self.activation = nn.ReLU()
        self.flatten = nn.Flatten()
        self.pool = nn.MaxPool2d(2, 2)
        # self.dropout25 = nn.Dropout(0.25)
        self.dropout50 = nn.Dropout(0.5)
        
    def forward(self, x):
        x = self.pool(self.activation(self.bn1(self.conv1(x))))
        x = self.pool(self.activation(self.bn2(self.conv2(x))))
        x = self.pool(self.activation(self.bn3(self.conv3(x))))
        x = self.attention1(x)
        x = self.dropout50(self.flatten(x))
        x = self.dropout50(self.activation(self.fc1(x)))
        x = self.fc2(x)
        return x


for lr in [1e-5]:
    model = CNNModel(num_classes=50).to(device)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.AdamW(model.parameters(), lr=lr, weight_decay=1e-2)
    num_epochs = 10
    save_path = 'record/Q2_less_layers'
    rec_name = '6layers_2048_dropout50_adamW'
    model_path = f'{save_path}/model_weight/model_weight_lr{lr}_{rec_name}.pth'
    best_val_loss = float('inf')
    best_model = None
    train_loss = []
    train_acc = []
    val_loss = []
    val_acc = []
    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0
        running_acc = 0.0
        pbar = tqdm(data_loader_train, colour='green', total=len(data_loader_train))
        for images, labels in pbar:
            images, labels = images.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()*len(labels)
            running_acc += (outputs.argmax(dim=1) == labels).float().sum().item()
            pbar.set_description(f'Train Iter: {epoch+1:03}/{num_epochs:03}  Loss: {loss:3.4f}')
            pbar.update()
        running_loss_avg = running_loss / len(dataset_train)
        running_acc_avg = running_acc / len(dataset_train)
        train_loss.append(running_loss_avg)
        train_acc.append(running_acc_avg)
        print(f"Epoch {epoch+1}, Loss: {running_loss_avg}, Accuracy: {running_acc_avg}")

        model.eval()
        with torch.no_grad():
            class_counts = [0] * 50
            running_loss = 0.0
            running_acc = 0.0
            pbar = tqdm(data_loader_val, colour='red', total=len(data_loader_val))
            for images, labels in pbar:
                images, labels = images.to(device), labels.to(device)
                outputs = model(images)
                predictions = outputs.argmax(dim=1)
                loss = criterion(outputs, labels)
                running_loss += loss.item()*len(labels)
                running_acc += (predictions == labels).float().sum().item()
                pbar.set_description(f'Valid Iter: {epoch+1:03}/{num_epochs:03}  Loss: {loss:3.4f}')
                pbar.update()

                for pred_label in predictions:
                    class_counts[pred_label] += 1

            running_loss_avg = running_loss / len(dataset_val)
            running_acc_avg = running_acc / len(dataset_val)
            val_loss.append(running_loss_avg)
            val_acc.append(running_acc_avg)
            print(f"Validation Loss: {running_loss_avg}, Accuracy: {running_acc_avg}")
            print(f"Class Counts: {class_counts}")
            if running_loss_avg < best_val_loss:
                best_val_loss = running_loss
                best_model = model
                torch.save(model.state_dict(), model_path)
        print("======================================")
    print("Finished Training")


    model.eval()
    with torch.no_grad():
        class_counts = [0] * 50
        running_loss = 0.0
        running_acc = 0.0
        predictions = []
        true_labels = []
        pbar = tqdm(data_loader_test, colour='red', total=len(data_loader_test))
        for images, labels in pbar:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)

            _, predicted = torch.max(outputs, 1)
            predictions.extend(predicted.cpu().numpy())
            true_labels.extend(labels.cpu().numpy())
            loss = criterion(outputs, labels)

            running_loss += loss.item()*len(labels)
            running_acc += (outputs.argmax(dim=1) == labels).float().sum().item()
            pbar.set_description(f'Test Iter: {epoch+1:03}/{num_epochs:03}  Loss: {loss:3.4f}')
            pbar.update()

        for pred_label in predictions:
            class_counts[pred_label] += 1
        
        precision = precision_score(true_labels, predictions, average='weighted')
        recall = recall_score(true_labels, predictions, average='weighted')
        f1 = f1_score(true_labels, predictions, average='weighted')
        print(f"Test Loss: {running_loss/len(dataset_test)}, Accuracy: {running_acc/len(dataset_test)}")
        print(f"Precision: {precision}, Recall: {recall}, F1: {f1}")
        print(f"Class Counts: {class_counts}")
    print("Finished Testing")

    train_record = {"train_loss": train_loss, "train_acc": train_acc, "val_loss": val_loss, "val_acc": val_acc}
    test_record = {"true_labels": true_labels, "predictions": predictions, "precision": precision, "recall": recall, "f1": f1, "class_counts": class_counts}


    with open(f'{save_path}/train_record/train_record_lr{lr}_{rec_name}.pkl', 'wb') as f:
        pickle.dump(train_record, f)
    with open(f'{save_path}/test_record/test_record_lr{lr}_{rec_name}.pkl', 'wb') as f:
        pickle.dump(test_record, f)


Train Iter: 001/010  Loss: 3.2933: 100%|██████████| 1979/1979 [03:44<00:00,  8.82it/s]


Epoch 1, Loss: 3.371419522149545, Accuracy: 0.1318278720884327


Valid Iter: 001/010  Loss: 2.0642: 100%|██████████| 15/15 [00:00<00:00, 25.23it/s]


Validation Loss: 2.968172470728556, Accuracy: 0.21777777777777776
Class Counts: [3, 10, 5, 23, 19, 18, 2, 6, 4, 0, 0, 4, 0, 3, 9, 4, 1, 10, 1, 0, 17, 16, 9, 21, 1, 4, 2, 14, 22, 10, 4, 5, 3, 11, 25, 7, 12, 12, 4, 22, 9, 5, 0, 24, 10, 11, 13, 5, 20, 10]


Train Iter: 002/010  Loss: 3.2808: 100%|██████████| 1979/1979 [03:44<00:00,  8.81it/s]


...

Epoch 10, Loss: 0.7263026210473923, Accuracy: 0.8228819581523885


Valid Iter: 010/010  Loss: 0.1971: 100%|██████████| 15/15 [00:00<00:00, 25.04it/s]


Validation Loss: 2.368212363587485, Accuracy: 0.36444444444444446
Class Counts: [7, 10, 11, 9, 23, 9, 4, 12, 13, 7, 4, 6, 5, 7, 13, 16, 7, 15, 8, 11, 12, 5, 5, 6, 4, 7, 6, 10, 6, 8, 12, 5, 7, 10, 9, 8, 14, 6, 7, 10, 12, 9, 7, 10, 8, 9, 11, 8, 10, 12]
Finished Training


Test Iter: 010/010  Loss: 0.1971: 100%|██████████| 15/15 [00:00<00:00, 25.06it/s]

Test Loss: 0.1970612108707428, Accuracy: 0.36666666666666664
Precision: 0.3737601581425111, Recall: 0.36666666666666664, F1: 0.3616002912897515
Class Counts: [9, 12, 15, 6, 17, 10, 7, 12, 13, 10, 3, 6, 4, 10, 12, 14, 8, 10, 10, 9, 7, 5, 8, 3, 6, 12, 4, 7, 12, 8, 11, 7, 5, 12, 10, 8, 6, 7, 13, 7, 8, 10, 11, 7, 9, 10, 8, 11, 10, 11]
Finished Testing


In [3]:
if torch.cuda.is_available():
    device = torch.device("cuda")
else:
    device = torch.device("cpu")
class SelfAttention(nn.Module):
    def __init__(self, in_channels):
        super(SelfAttention, self).__init__()
        self.query_conv = nn.Conv2d(in_channels, in_channels // 8, kernel_size=1)
        self.key_conv = nn.Conv2d(in_channels, in_channels // 8, kernel_size=1)
        self.value_conv = nn.Conv2d(in_channels, in_channels, kernel_size=1)
        self.gamma = nn.Parameter(torch.zeros(1))
        
    def forward(self, x):
        batch_size, C, width, height = x.size()
        
        # 计算查询、键和值
        proj_query = self.query_conv(x).view(batch_size, -1, width * height).permute(0, 2, 1)
        proj_key = self.key_conv(x).view(batch_size, -1, width * height)
        energy = torch.bmm(proj_query, proj_key)
        
        # 计算注意力权重并应用到值上
        attention = F.softmax(energy, dim=-1)
        proj_value = self.value_conv(x).view(batch_size, -1, width * height)
        out = torch.bmm(proj_value, attention.permute(0, 2, 1))
        
        # 调整输出并加上残差连接
        out = out.view(batch_size, C, width, height)
        out = self.gamma * out + x
        return out

class CNNModel(nn.Module):
    def __init__(self, num_classes=50):
        super(CNNModel, self).__init__()
        self.conv1 = nn.Conv2d(3, 64, kernel_size=3, padding=1)
        self.bn1 = nn.BatchNorm2d(64)
        self.conv2 = nn.Conv2d(64, 128, kernel_size=3, padding=1)
        self.bn2 = nn.BatchNorm2d(128)
        self.conv3 = nn.Conv2d(128, 256, kernel_size=3, padding=1)
        self.bn3 = nn.BatchNorm2d(256)
        self.attention1 = SelfAttention(256)

        self.fc1 = nn.Linear(256*28*28, 2048)
        self.fc2 = nn.Linear(2048, num_classes)
        self.activation = nn.ReLU()
        self.flatten = nn.Flatten()
        self.pool = nn.MaxPool2d(2, 2)
        # self.dropout25 = nn.Dropout(0.25)
        self.dropout50 = nn.Dropout(0.5)
        
    def forward(self, x):
        x = self.pool(self.activation(self.bn1(self.conv1(x))))
        x = self.pool(self.activation(self.bn2(self.conv2(x))))
        x = self.pool(self.activation(self.bn3(self.conv3(x))))
        x = self.attention1(x)
        x = self.dropout50(self.flatten(x))
        x = self.dropout50(self.activation(self.fc1(x)))
        x = self.fc2(x)
        return x


for lr in [1e-5]:
    model = CNNModel(num_classes=50).to(device)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.AdamW(model.parameters(), lr=lr, weight_decay=0.00025)
    num_epochs = 10
    save_path = 'record/Q2_less_layers'
    rec_name = '6layers_2048_dropout50_adamW_0.00025'
    model_path = f'{save_path}/model_weight/model_weight_lr{lr}_{rec_name}.pth'
    best_val_loss = float('inf')
    best_model = None
    train_loss = []
    train_acc = []
    val_loss = []
    val_acc = []
    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0
        running_acc = 0.0
        pbar = tqdm(data_loader_train, colour='green', total=len(data_loader_train))
        for images, labels in pbar:
            images, labels = images.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()*len(labels)
            running_acc += (outputs.argmax(dim=1) == labels).float().sum().item()
            pbar.set_description(f'Train Iter: {epoch+1:03}/{num_epochs:03}  Loss: {loss:3.4f}')
            pbar.update()
        running_loss_avg = running_loss / len(dataset_train)
        running_acc_avg = running_acc / len(dataset_train)
        train_loss.append(running_loss_avg)
        train_acc.append(running_acc_avg)
        print(f"Epoch {epoch+1}, Loss: {running_loss_avg}, Accuracy: {running_acc_avg}")

        model.eval()
        with torch.no_grad():
            class_counts = [0] * 50
            running_loss = 0.0
            running_acc = 0.0
            pbar = tqdm(data_loader_val, colour='red', total=len(data_loader_val))
            for images, labels in pbar:
                images, labels = images.to(device), labels.to(device)
                outputs = model(images)
                predictions = outputs.argmax(dim=1)
                loss = criterion(outputs, labels)
                running_loss += loss.item()*len(labels)
                running_acc += (predictions == labels).float().sum().item()
                pbar.set_description(f'Valid Iter: {epoch+1:03}/{num_epochs:03}  Loss: {loss:3.4f}')
                pbar.update()

                for pred_label in predictions:
                    class_counts[pred_label] += 1

            running_loss_avg = running_loss / len(dataset_val)
            running_acc_avg = running_acc / len(dataset_val)
            val_loss.append(running_loss_avg)
            val_acc.append(running_acc_avg)
            print(f"Validation Loss: {running_loss_avg}, Accuracy: {running_acc_avg}")
            print(f"Class Counts: {class_counts}")
            if running_loss_avg < best_val_loss:
                best_val_loss = running_loss
                best_model = model
                torch.save(model.state_dict(), model_path)
        print("======================================")
    print("Finished Training")


    model.eval()
    with torch.no_grad():
        class_counts = [0] * 50
        running_loss = 0.0
        running_acc = 0.0
        predictions = []
        true_labels = []
        pbar = tqdm(data_loader_test, colour='red', total=len(data_loader_test))
        for images, labels in pbar:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)

            _, predicted = torch.max(outputs, 1)
            predictions.extend(predicted.cpu().numpy())
            true_labels.extend(labels.cpu().numpy())
            loss = criterion(outputs, labels)

            running_loss += loss.item()*len(labels)
            running_acc += (outputs.argmax(dim=1) == labels).float().sum().item()
            pbar.set_description(f'Test Iter: {epoch+1:03}/{num_epochs:03}  Loss: {loss:3.4f}')
            pbar.update()

        for pred_label in predictions:
            class_counts[pred_label] += 1
        
        precision = precision_score(true_labels, predictions, average='weighted')
        recall = recall_score(true_labels, predictions, average='weighted')
        f1 = f1_score(true_labels, predictions, average='weighted')
        print(f"Test Loss: {running_loss/len(dataset_test)}, Accuracy: {running_acc/len(dataset_test)}")
        print(f"Precision: {precision}, Recall: {recall}, F1: {f1}")
        print(f"Class Counts: {class_counts}")
    print("Finished Testing")

    train_record = {"train_loss": train_loss, "train_acc": train_acc, "val_loss": val_loss, "val_acc": val_acc}
    test_record = {"true_labels": true_labels, "predictions": predictions, "precision": precision, "recall": recall, "f1": f1, "class_counts": class_counts}


    with open(f'{save_path}/train_record/train_record_lr{lr}_{rec_name}.pkl', 'wb') as f:
        pickle.dump(train_record, f)
    with open(f'{save_path}/test_record/test_record_lr{lr}_{rec_name}.pkl', 'wb') as f:
        pickle.dump(test_record, f)


Train Iter: 001/010  Loss: 3.0217: 100%|██████████| 1979/1979 [03:48<00:00,  8.65it/s]


Epoch 1, Loss: 3.3622765291381906, Accuracy: 0.13136991709435453


Valid Iter: 001/010  Loss: 2.1082: 100%|██████████| 15/15 [00:00<00:00, 25.22it/s]


Validation Loss: 2.963844184875488, Accuracy: 0.18666666666666668
Class Counts: [14, 5, 22, 7, 22, 5, 9, 6, 2, 4, 0, 0, 0, 3, 9, 16, 4, 0, 10, 1, 27, 2, 6, 15, 9, 1, 1, 17, 3, 11, 5, 0, 20, 11, 10, 13, 11, 13, 2, 24, 12, 6, 13, 14, 7, 13, 10, 4, 18, 13]


Train Iter: 002/010  Loss: 2.7883: 100%|██████████| 1979/1979 [03:47<00:00,  8.69it/s]


...

Epoch 10, Loss: 0.6981750429961411, Accuracy: 0.8290248716936439


Valid Iter: 010/010  Loss: 0.3383: 100%|██████████| 15/15 [00:00<00:00, 24.91it/s]


Validation Loss: 2.3225330283906724, Accuracy: 0.35333333333333333
Class Counts: [10, 8, 7, 16, 21, 10, 3, 7, 6, 8, 9, 3, 6, 11, 8, 5, 7, 9, 8, 12, 16, 4, 10, 8, 4, 3, 12, 10, 5, 12, 7, 8, 10, 12, 6, 11, 10, 13, 9, 10, 13, 5, 9, 14, 9, 10, 9, 6, 12, 9]
Finished Training


Test Iter: 010/010  Loss: 0.3383: 100%|██████████| 15/15 [00:00<00:00, 24.42it/s]

Test Loss: 0.3383125066757202, Accuracy: 0.38
Precision: 0.39352695017400896, Recall: 0.38, F1: 0.3740236531390118
Class Counts: [6, 8, 11, 11, 12, 12, 8, 7, 5, 4, 6, 4, 6, 17, 9, 11, 12, 11, 14, 11, 11, 3, 8, 10, 8, 7, 4, 9, 10, 11, 5, 12, 6, 10, 9, 11, 3, 11, 11, 11, 8, 9, 12, 5, 8, 14, 12, 6, 10, 11]
Finished Testing


In [4]:
with open('images/images.pickle', 'rb') as f:
    img_data = pickle.load(f)
class CustomDataset(Dataset):
    def __init__(self, img_data, txt_file, transform=None):
        self.data = img_data
        with open(txt_file, 'r') as f:
            self.labels = f.readlines()
        self.transform = transform
    def __len__(self):
        return len(self.labels)
    def __getitem__(self, idx):
        img_path, label = self.labels[idx].strip().split()
        img_path = img_path.split('/')
        img = self.data[img_path[1]][img_path[2]]
        img = Image.open(io.BytesIO(img)).convert('RGB')
        if self.transform:
            img = self.transform(img)
        label = torch.tensor(int(label))
        return img, label

transform_train = transforms.Compose([
    transforms.RandomResizedCrop(224),
    transforms.RandomHorizontalFlip(),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),
    transforms.RandomRotation(degrees=10),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])


transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])
dataset_train = CustomDataset(img_data=img_data, txt_file='images/train.txt', transform=transform_train)
dataset_val = CustomDataset(img_data=img_data, txt_file='images/val.txt', transform=transform)
dataset_test = CustomDataset(img_data=img_data, txt_file='images/test.txt', transform=transform)


data_loader_train = DataLoader(dataset_train, batch_size=32, shuffle=True, )
data_loader_val = DataLoader(dataset_val, batch_size=32, shuffle=False)
data_loader_test = DataLoader(dataset_test, batch_size=32, shuffle=False)
if torch.cuda.is_available():
    device = torch.device("cuda")
else:
    device = torch.device("cpu")
class SelfAttention(nn.Module):
    def __init__(self, in_channels):
        super(SelfAttention, self).__init__()
        self.query_conv = nn.Conv2d(in_channels, in_channels // 8, kernel_size=1)
        self.key_conv = nn.Conv2d(in_channels, in_channels // 8, kernel_size=1)
        self.value_conv = nn.Conv2d(in_channels, in_channels, kernel_size=1)
        self.gamma = nn.Parameter(torch.zeros(1))
        
    def forward(self, x):
        batch_size, C, width, height = x.size()
        
        # 计算查询、键和值
        proj_query = self.query_conv(x).view(batch_size, -1, width * height).permute(0, 2, 1)
        proj_key = self.key_conv(x).view(batch_size, -1, width * height)
        energy = torch.bmm(proj_query, proj_key)
        
        # 计算注意力权重并应用到值上
        attention = F.softmax(energy, dim=-1)
        proj_value = self.value_conv(x).view(batch_size, -1, width * height)
        out = torch.bmm(proj_value, attention.permute(0, 2, 1))
        
        # 调整输出并加上残差连接
        out = out.view(batch_size, C, width, height)
        out = self.gamma * out + x
        return out

class CNNModel(nn.Module):
    def __init__(self, num_classes=50):
        super(CNNModel, self).__init__()
        self.conv1 = nn.Conv2d(3, 64, kernel_size=3, padding=1)
        self.bn1 = nn.BatchNorm2d(64)
        self.conv2 = nn.Conv2d(64, 128, kernel_size=3, padding=1)
        self.bn2 = nn.BatchNorm2d(128)
        self.conv3 = nn.Conv2d(128, 256, kernel_size=3, padding=1)
        self.bn3 = nn.BatchNorm2d(256)
        self.attention1 = SelfAttention(256)

        self.fc1 = nn.Linear(256*28*28, 2048)
        self.fc2 = nn.Linear(2048, num_classes)
        self.activation = nn.ReLU()
        self.flatten = nn.Flatten()
        self.pool = nn.MaxPool2d(2, 2)
        # self.dropout25 = nn.Dropout(0.25)
        self.dropout50 = nn.Dropout(0.5)
        
    def forward(self, x):
        x = self.pool(self.activation(self.bn1(self.conv1(x))))
        x = self.pool(self.activation(self.bn2(self.conv2(x))))
        x = self.pool(self.activation(self.bn3(self.conv3(x))))
        x = self.attention1(x)
        x = self.dropout50(self.flatten(x))
        x = self.dropout50(self.activation(self.fc1(x)))
        x = self.fc2(x)
        return x


for lr in [1e-5]:
    model = CNNModel(num_classes=50).to(device)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.AdamW(model.parameters(), lr=lr, weight_decay=0.00025)
    num_epochs = 10
    save_path = 'record/Q2_less_layers'
    rec_name = '6layers_2048_dropout50_adamW_0.00025_augment'
    model_path = f'{save_path}/model_weight/model_weight_lr{lr}_{rec_name}.pth'
    best_val_loss = float('inf')
    best_model = None
    train_loss = []
    train_acc = []
    val_loss = []
    val_acc = []
    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0
        running_acc = 0.0
        pbar = tqdm(data_loader_train, colour='green', total=len(data_loader_train))
        for images, labels in pbar:
            images, labels = images.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()*len(labels)
            running_acc += (outputs.argmax(dim=1) == labels).float().sum().item()
            pbar.set_description(f'Train Iter: {epoch+1:03}/{num_epochs:03}  Loss: {loss:3.4f}')
            pbar.update()
        running_loss_avg = running_loss / len(dataset_train)
        running_acc_avg = running_acc / len(dataset_train)
        train_loss.append(running_loss_avg)
        train_acc.append(running_acc_avg)
        print(f"Epoch {epoch+1}, Loss: {running_loss_avg}, Accuracy: {running_acc_avg}")

        model.eval()
        with torch.no_grad():
            class_counts = [0] * 50
            running_loss = 0.0
            running_acc = 0.0
            pbar = tqdm(data_loader_val, colour='red', total=len(data_loader_val))
            for images, labels in pbar:
                images, labels = images.to(device), labels.to(device)
                outputs = model(images)
                predictions = outputs.argmax(dim=1)
                loss = criterion(outputs, labels)
                running_loss += loss.item()*len(labels)
                running_acc += (predictions == labels).float().sum().item()
                pbar.set_description(f'Valid Iter: {epoch+1:03}/{num_epochs:03}  Loss: {loss:3.4f}')
                pbar.update()

                for pred_label in predictions:
                    class_counts[pred_label] += 1

            running_loss_avg = running_loss / len(dataset_val)
            running_acc_avg = running_acc / len(dataset_val)
            val_loss.append(running_loss_avg)
            val_acc.append(running_acc_avg)
            print(f"Validation Loss: {running_loss_avg}, Accuracy: {running_acc_avg}")
            print(f"Class Counts: {class_counts}")
            if running_loss_avg < best_val_loss:
                best_val_loss = running_loss
                best_model = model
                torch.save(model.state_dict(), model_path)
        print("======================================")
    print("Finished Training")


    model.eval()
    with torch.no_grad():
        class_counts = [0] * 50
        running_loss = 0.0
        running_acc = 0.0
        predictions = []
        true_labels = []
        pbar = tqdm(data_loader_test, colour='red', total=len(data_loader_test))
        for images, labels in pbar:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)

            _, predicted = torch.max(outputs, 1)
            predictions.extend(predicted.cpu().numpy())
            true_labels.extend(labels.cpu().numpy())
            loss = criterion(outputs, labels)

            running_loss += loss.item()*len(labels)
            running_acc += (outputs.argmax(dim=1) == labels).float().sum().item()
            pbar.set_description(f'Test Iter: {epoch+1:03}/{num_epochs:03}  Loss: {loss:3.4f}')
            pbar.update()

        for pred_label in predictions:
            class_counts[pred_label] += 1
        
        precision = precision_score(true_labels, predictions, average='weighted')
        recall = recall_score(true_labels, predictions, average='weighted')
        f1 = f1_score(true_labels, predictions, average='weighted')
        print(f"Test Loss: {running_loss/len(dataset_test)}, Accuracy: {running_acc/len(dataset_test)}")
        print(f"Precision: {precision}, Recall: {recall}, F1: {f1}")
        print(f"Class Counts: {class_counts}")
    print("Finished Testing")

    train_record = {"train_loss": train_loss, "train_acc": train_acc, "val_loss": val_loss, "val_acc": val_acc}
    test_record = {"true_labels": true_labels, "predictions": predictions, "precision": precision, "recall": recall, "f1": f1, "class_counts": class_counts}


    with open(f'{save_path}/train_record/train_record_lr{lr}_{rec_name}.pkl', 'wb') as f:
        pickle.dump(train_record, f)
    with open(f'{save_path}/test_record/test_record_lr{lr}_{rec_name}.pkl', 'wb') as f:
        pickle.dump(test_record, f)


Train Iter: 001/010  Loss: 3.6530: 100%|██████████| 1979/1979 [06:31<00:00,  5.06it/s]


Epoch 1, Loss: 3.7190854269367315, Accuracy: 0.0704776944334781


Valid Iter: 001/010  Loss: 2.6371: 100%|██████████| 15/15 [00:00<00:00, 20.62it/s]


Validation Loss: 3.3658720927768284, Accuracy: 0.13111111111111112
Class Counts: [14, 11, 17, 6, 22, 0, 0, 2, 0, 1, 0, 0, 0, 1, 8, 12, 3, 0, 7, 0, 16, 2, 0, 3, 0, 1, 0, 5, 24, 0, 0, 0, 103, 28, 16, 21, 11, 14, 1, 6, 3, 2, 0, 5, 8, 12, 3, 3, 16, 43]


Train Iter: 002/010  Loss: 3.7509: 100%|██████████| 1979/1979 [06:39<00:00,  4.96it/s]


...

Valid Iter: 010/010  Loss: 1.9690: 100%|██████████| 15/15 [00:00<00:00, 21.01it/s]


Validation Loss: 2.749142740037706, Accuracy: 0.22
Class Counts: [6, 18, 6, 5, 25, 8, 5, 3, 2, 0, 2, 2, 1, 13, 6, 11, 3, 3, 1, 6, 25, 10, 11, 8, 5, 20, 7, 16, 8, 2, 8, 13, 12, 3, 6, 14, 7, 10, 9, 21, 14, 6, 11, 12, 7, 7, 7, 9, 19, 17]
Finished Training


Test Iter: 010/010  Loss: 1.9690: 100%|██████████| 15/15 [00:00<00:00, 20.90it/s]

Test Loss: 1.9690446853637695, Accuracy: 0.2688888888888889
Precision: 0.2634189452805013, Recall: 0.2688888888888889, F1: 0.24570814122970888
Class Counts: [8, 15, 5, 9, 10, 14, 3, 7, 4, 0, 1, 2, 0, 14, 4, 14, 5, 1, 1, 7, 18, 8, 11, 7, 2, 22, 3, 15, 11, 4, 9, 13, 7, 13, 8, 7, 6, 7, 18, 15, 23, 3, 11, 6, 10, 10, 13, 9, 18, 19]
Finished Testing



/opt/conda/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [15]:
with open('images/images.pickle', 'rb') as f:
    img_data = pickle.load(f)
class CustomDataset(Dataset):
    def __init__(self, img_data, txt_file, transform=None):
        self.data = img_data
        with open(txt_file, 'r') as f:
            self.labels = f.readlines()
        self.transform = transform
    def __len__(self):
        return len(self.labels)
    def __getitem__(self, idx):
        img_path, label = self.labels[idx].strip().split()
        img_path = img_path.split('/')
        img = self.data[img_path[1]][img_path[2]]
        img = Image.open(io.BytesIO(img)).convert('RGB')
        if self.transform:
            img = self.transform(img)
        label = torch.tensor(int(label))
        return img, label

transform_train = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.ColorJitter(brightness=0.1, contrast=0.2, saturation=0.2, hue=0),
    transforms.RandomRotation(degrees=10),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])


transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])
dataset_train = CustomDataset(img_data=img_data, txt_file='images/train.txt', transform=transform_train)
dataset_val = CustomDataset(img_data=img_data, txt_file='images/val.txt', transform=transform)
dataset_test = CustomDataset(img_data=img_data, txt_file='images/test.txt', transform=transform)


data_loader_train = DataLoader(dataset_train, batch_size=32, shuffle=True, num_workers=16)
data_loader_val = DataLoader(dataset_val, batch_size=32, shuffle=False, num_workers=16)
data_loader_test = DataLoader(dataset_test, batch_size=32, shuffle=False, num_workers=16)
if torch.cuda.is_available():
    device = torch.device("cuda")
else:
    device = torch.device("cpu")
class SelfAttention(nn.Module):
    def __init__(self, in_channels):
        super(SelfAttention, self).__init__()
        self.query_conv = nn.Conv2d(in_channels, in_channels // 8, kernel_size=1)
        self.key_conv = nn.Conv2d(in_channels, in_channels // 8, kernel_size=1)
        self.value_conv = nn.Conv2d(in_channels, in_channels, kernel_size=1)
        self.gamma = nn.Parameter(torch.zeros(1))
        
    def forward(self, x):
        batch_size, C, width, height = x.size()
        
        # 计算查询、键和值
        proj_query = self.query_conv(x).view(batch_size, -1, width * height).permute(0, 2, 1)
        proj_key = self.key_conv(x).view(batch_size, -1, width * height)
        energy = torch.bmm(proj_query, proj_key)
        
        # 计算注意力权重并应用到值上
        attention = F.softmax(energy, dim=-1)
        proj_value = self.value_conv(x).view(batch_size, -1, width * height)
        out = torch.bmm(proj_value, attention.permute(0, 2, 1))
        
        # 调整输出并加上残差连接
        out = out.view(batch_size, C, width, height)
        out = self.gamma * out + x
        return out

class CNNModel(nn.Module):
    def __init__(self, num_classes=50):
        super(CNNModel, self).__init__()
        self.conv1 = nn.Conv2d(3, 64, kernel_size=3, padding=1)
        self.bn1 = nn.BatchNorm2d(64)
        self.conv2 = nn.Conv2d(64, 128, kernel_size=3, padding=1)
        self.bn2 = nn.BatchNorm2d(128)
        self.conv3 = nn.Conv2d(128, 256, kernel_size=3, padding=1)
        self.bn3 = nn.BatchNorm2d(256)
        self.attention1 = SelfAttention(256)

        self.fc1 = nn.Linear(256*28*28, 2048)
        self.fc2 = nn.Linear(2048, num_classes)
        self.activation = nn.ReLU()
        self.flatten = nn.Flatten()
        self.pool = nn.MaxPool2d(2, 2)
        # self.dropout25 = nn.Dropout(0.25)
        self.dropout50 = nn.Dropout(0.5)
        
    def forward(self, x):
        x = self.pool(self.activation(self.bn1(self.conv1(x))))
        x = self.pool(self.activation(self.bn2(self.conv2(x))))
        x = self.pool(self.activation(self.bn3(self.conv3(x))))
        x = self.attention1(x)
        x = self.dropout50(self.flatten(x))
        x = self.dropout50(self.activation(self.fc1(x)))
        x = self.fc2(x)
        return x


for lr in [1e-4, 1e-5, 1e-6]:
    model = CNNModel(num_classes=50).to(device)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.AdamW(model.parameters(), lr=lr, weight_decay=0.00025)
    num_epochs = 10
    save_path = 'record/Q2_less_layers'
    rec_name = '6layers_2048_dropout50_adamW_0.00025_augmentless'
    model_path = f'{save_path}/model_weight/model_weight_lr{lr}_{rec_name}.pth'
    best_val_loss = float('inf')
    best_model = None
    train_loss = []
    train_acc = []
    val_loss = []
    val_acc = []
    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0
        running_acc = 0.0
        pbar = tqdm(data_loader_train, colour='green', total=len(data_loader_train))
        for images, labels in pbar:
            images, labels = images.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()*len(labels)
            running_acc += (outputs.argmax(dim=1) == labels).float().sum().item()
            pbar.set_description(f'Train Iter: {epoch+1:03}/{num_epochs:03}  Loss: {loss:3.4f}')
            pbar.update()
        running_loss_avg = running_loss / len(dataset_train)
        running_acc_avg = running_acc / len(dataset_train)
        train_loss.append(running_loss_avg)
        train_acc.append(running_acc_avg)
        print(f"Epoch {epoch+1}, Loss: {running_loss_avg}, Accuracy: {running_acc_avg}")

        model.eval()
        with torch.no_grad():
            class_counts = [0] * 50
            running_loss = 0.0
            running_acc = 0.0
            pbar = tqdm(data_loader_val, colour='red', total=len(data_loader_val))
            for images, labels in pbar:
                images, labels = images.to(device), labels.to(device)
                outputs = model(images)
                predictions = outputs.argmax(dim=1)
                loss = criterion(outputs, labels)
                running_loss += loss.item()*len(labels)
                running_acc += (predictions == labels).float().sum().item()
                pbar.set_description(f'Valid Iter: {epoch+1:03}/{num_epochs:03}  Loss: {loss:3.4f}')
                pbar.update()

                for pred_label in predictions:
                    class_counts[pred_label] += 1

            running_loss_avg = running_loss / len(dataset_val)
            running_acc_avg = running_acc / len(dataset_val)
            val_loss.append(running_loss_avg)
            val_acc.append(running_acc_avg)
            print(f"Validation Loss: {running_loss_avg}, Accuracy: {running_acc_avg}")
            print(f"Class Counts: {class_counts}")
            if running_loss_avg < best_val_loss:
                best_val_loss = running_loss
                best_model = model
                torch.save(model.state_dict(), model_path)
        print("======================================")
    print("Finished Training")


    model.eval()
    with torch.no_grad():
        class_counts = [0] * 50
        running_loss = 0.0
        running_acc = 0.0
        predictions = []
        true_labels = []
        pbar = tqdm(data_loader_test, colour='red', total=len(data_loader_test))
        for images, labels in pbar:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)

            _, predicted = torch.max(outputs, 1)
            predictions.extend(predicted.cpu().numpy())
            true_labels.extend(labels.cpu().numpy())
            loss = criterion(outputs, labels)
            
            running_loss += loss.item()*len(labels)
            running_acc += (outputs.argmax(dim=1) == labels).float().sum().item()
            pbar.set_description(f'Test Iter: {epoch+1:03}/{num_epochs:03}  Loss: {loss:3.4f}')
            pbar.update()

        for pred_label in predictions:
            class_counts[pred_label] += 1
        
        precision = precision_score(true_labels, predictions, average='weighted')
        recall = recall_score(true_labels, predictions, average='weighted')
        f1 = f1_score(true_labels, predictions, average='weighted')
        print(f"Test Loss: {running_loss/len(dataset_test)}, Accuracy: {running_acc/len(dataset_test)}")
        print(f"Precision: {precision}, Recall: {recall}, F1: {f1}")
        print(f"Class Counts: {class_counts}")
    print("Finished Testing")

    train_record = {"train_loss": train_loss, "train_acc": train_acc, "val_loss": val_loss, "val_acc": val_acc}
    test_record = {"true_labels": true_labels, "predictions": predictions, "precision": precision, "recall": recall, "f1": f1, "class_counts": class_counts}


    with open(f'{save_path}/train_record/train_record_lr{lr}_{rec_name}.pkl', 'wb') as f:
        pickle.dump(train_record, f)
    with open(f'{save_path}/test_record/test_record_lr{lr}_{rec_name}.pkl', 'wb') as f:
        pickle.dump(test_record, f)


Train Iter: 001/010  Loss: 3.3986: 100%|██████████| 1979/1979 [02:55<00:00, 11.26it/s]


Epoch 1, Loss: 3.7106129950061413, Accuracy: 0.08521121200157916


Valid Iter: 001/010  Loss: 2.2721: 100%|██████████| 15/15 [00:01<00:00, 10.24it/s]


Validation Loss: 3.1425390021006265, Accuracy: 0.14666666666666667
Class Counts: [14, 3, 13, 6, 25, 10, 6, 5, 0, 3, 0, 0, 0, 3, 2, 7, 11, 1, 5, 25, 3, 1, 17, 10, 3, 4, 4, 8, 26, 11, 3, 3, 24, 13, 12, 15, 0, 18, 5, 12, 2, 18, 16, 9, 8, 13, 19, 6, 23, 5]


Train Iter: 002/010  Loss: 2.9060: 100%|██████████| 1979/1979 [02:55<00:00, 11.26it/s]


...

Epoch 10, Loss: 2.794634328654666, Accuracy: 0.24709040663245163


Valid Iter: 010/010  Loss: 0.6913: 100%|██████████| 15/15 [00:01<00:00, 10.67it/s]


Validation Loss: 2.665474801858266, Accuracy: 0.27555555555555555
Class Counts: [3, 5, 17, 5, 21, 13, 9, 6, 12, 1, 5, 1, 6, 4, 5, 6, 2, 8, 5, 14, 8, 9, 15, 5, 3, 11, 1, 13, 5, 5, 5, 8, 5, 18, 15, 11, 18, 8, 10, 11, 18, 13, 1, 17, 11, 10, 7, 3, 17, 21]
Finished Training


Test Iter: 010/010  Loss: 0.6913: 100%|██████████| 15/15 [00:01<00:00, 10.91it/s]


Test Loss: 0.6913202404975891, Accuracy: 0.29555555555555557
Precision: 0.3302085315785006, Recall: 0.29555555555555557, F1: 0.28583110524280264
Class Counts: [12, 8, 14, 10, 9, 14, 4, 6, 12, 1, 5, 1, 6, 3, 2, 10, 6, 9, 10, 6, 9, 7, 19, 11, 2, 8, 8, 17, 7, 9, 2, 4, 5, 21, 15, 11, 15, 5, 7, 14, 17, 11, 2, 7, 9, 17, 9, 7, 9, 18]
Finished Testing


In [20]:
with open('images/images.pickle', 'rb') as f:
    img_data = pickle.load(f)
class CustomDataset(Dataset):
    def __init__(self, img_data, txt_file, transform=None):
        self.data = img_data
        with open(txt_file, 'r') as f:
            self.labels = f.readlines()
        self.transform = transform
    def __len__(self):
        return len(self.labels)
    def __getitem__(self, idx):
        img_path, label = self.labels[idx].strip().split()
        img_path = img_path.split('/')
        img = self.data[img_path[1]][img_path[2]]
        img = Image.open(io.BytesIO(img)).convert('RGB')
        if self.transform:
            img = self.transform(img)
        label = torch.tensor(int(label))
        return img, label

transform_train = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.ColorJitter(brightness=0.1, contrast=0.2, saturation=0.2, hue=0),
    transforms.RandomRotation(degrees=10),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])


transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])
dataset_train = CustomDataset(img_data=img_data, txt_file='images/train.txt', transform=transform_train)
dataset_val = CustomDataset(img_data=img_data, txt_file='images/val.txt', transform=transform)
dataset_test = CustomDataset(img_data=img_data, txt_file='images/test.txt', transform=transform)


data_loader_train = DataLoader(dataset_train, batch_size=32, shuffle=True, num_workers=16)
data_loader_val = DataLoader(dataset_val, batch_size=32, shuffle=False, num_workers=16)
data_loader_test = DataLoader(dataset_test, batch_size=32, shuffle=False, num_workers=16)
if torch.cuda.is_available():
    device = torch.device("cuda")
else:
    device = torch.device("cpu")
class SelfAttention(nn.Module):
    def __init__(self, in_channels):
        super(SelfAttention, self).__init__()
        self.query_conv = nn.Conv2d(in_channels, in_channels // 8, kernel_size=1)
        self.key_conv = nn.Conv2d(in_channels, in_channels // 8, kernel_size=1)
        self.value_conv = nn.Conv2d(in_channels, in_channels, kernel_size=1)
        self.gamma = nn.Parameter(torch.zeros(1))
        
    def forward(self, x):
        batch_size, C, width, height = x.size()
        
        # 计算查询、键和值
        proj_query = self.query_conv(x).view(batch_size, -1, width * height).permute(0, 2, 1)
        proj_key = self.key_conv(x).view(batch_size, -1, width * height)
        energy = torch.bmm(proj_query, proj_key)
        
        # 计算注意力权重并应用到值上
        attention = F.softmax(energy, dim=-1)
        proj_value = self.value_conv(x).view(batch_size, -1, width * height)
        out = torch.bmm(proj_value, attention.permute(0, 2, 1))
        
        # 调整输出并加上残差连接
        out = out.view(batch_size, C, width, height)
        out = self.gamma * out + x
        return out

class CNNModel(nn.Module):
    def __init__(self, num_classes=50):
        super(CNNModel, self).__init__()
        self.conv1 = nn.Conv2d(3, 64, kernel_size=3, padding=1)
        self.bn1 = nn.BatchNorm2d(64)
        self.conv2 = nn.Conv2d(64, 128, kernel_size=3, padding=1)
        self.bn2 = nn.BatchNorm2d(128)
        self.conv3 = nn.Conv2d(128, 256, kernel_size=3, padding=1)
        self.bn3 = nn.BatchNorm2d(256)
        self.attention1 = SelfAttention(256)

        self.fc1 = nn.Linear(256*28*28, 2048)
        self.fc2 = nn.Linear(2048, num_classes)
        self.activation = nn.ReLU()
        self.flatten = nn.Flatten()
        self.pool = nn.MaxPool2d(2, 2)
        # self.dropout25 = nn.Dropout(0.25)
        self.dropout50 = nn.Dropout(0.5)
        
    def forward(self, x):
        x = self.pool(self.activation(self.bn1(self.conv1(x))))
        x = self.pool(self.activation(self.bn2(self.conv2(x))))
        x = self.pool(self.activation(self.bn3(self.conv3(x))))
        x = self.attention1(x)
        x = self.dropout50(self.flatten(x))
        x = self.dropout50(self.activation(self.fc1(x)))
        x = self.fc2(x)
        return x


for lr in [1e-5]:
    model = CNNModel(num_classes=50).to(device)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.AdamW(model.parameters(), lr=lr, weight_decay=0.00025)
    num_epochs = 10
    save_path = 'record/Q2_less_layers'
    rec_name = '6layers_2048_dropout50_adamW_0.00025_augmentless_round2'
    model_path = f'{save_path}/model_weight/model_weight_lr{lr}_{rec_name}.pth'
    best_val_loss = float('inf')
    best_model = None
    train_loss = []
    train_acc = []
    val_loss = []
    val_acc = []
    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0
        running_acc = 0.0
        pbar = tqdm(data_loader_train, colour='green', total=len(data_loader_train))
        for images, labels in pbar:
            images, labels = images.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()*len(labels)
            running_acc += (outputs.argmax(dim=1) == labels).float().sum().item()
            pbar.set_description(f'Train Iter: {epoch+1:03}/{num_epochs:03}  Loss: {loss:3.4f}')
            pbar.update()
        running_loss_avg = running_loss / len(dataset_train)
        running_acc_avg = running_acc / len(dataset_train)
        train_loss.append(running_loss_avg)
        train_acc.append(running_acc_avg)
        print(f"Epoch {epoch+1}, Loss: {running_loss_avg}, Accuracy: {running_acc_avg}")

        model.eval()
        with torch.no_grad():
            class_counts = [0] * 50
            running_loss = 0.0
            running_acc = 0.0
            pbar = tqdm(data_loader_val, colour='red', total=len(data_loader_val))
            for images, labels in pbar:
                images, labels = images.to(device), labels.to(device)
                outputs = model(images)
                predictions = outputs.argmax(dim=1)
                loss = criterion(outputs, labels)
                running_loss += loss.item()*len(labels)
                running_acc += (predictions == labels).float().sum().item()
                pbar.set_description(f'Valid Iter: {epoch+1:03}/{num_epochs:03}  Loss: {loss:3.4f}')
                pbar.update()

                for pred_label in predictions:
                    class_counts[pred_label] += 1

            running_loss_avg = running_loss / len(dataset_val)
            running_acc_avg = running_acc / len(dataset_val)
            val_loss.append(running_loss_avg)
            val_acc.append(running_acc_avg)
            print(f"Validation Loss: {running_loss_avg}, Accuracy: {running_acc_avg}")
            print(f"Class Counts: {class_counts}")
            if running_loss_avg < best_val_loss:
                best_val_loss = running_loss
                best_model = model
                torch.save(model.state_dict(), model_path)
        print("======================================")
    print("Finished Training")


    model.eval()
    with torch.no_grad():
        class_counts = [0] * 50
        running_loss = 0.0
        running_acc = 0.0
        predictions = []
        true_labels = []
        pbar = tqdm(data_loader_test, colour='red', total=len(data_loader_test))
        for images, labels in pbar:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)

            _, predicted = torch.max(outputs, 1)
            predictions.extend(predicted.cpu().numpy())
            true_labels.extend(labels.cpu().numpy())
            loss = criterion(outputs, labels)

            running_loss += loss.item()*len(labels)
            running_acc += (outputs.argmax(dim=1) == labels).float().sum().item()
            pbar.set_description(f'Test Iter: {epoch+1:03}/{num_epochs:03}  Loss: {loss:3.4f}')
            pbar.update()

        for pred_label in predictions:
            class_counts[pred_label] += 1
        
        precision = precision_score(true_labels, predictions, average='weighted')
        recall = recall_score(true_labels, predictions, average='weighted')
        f1 = f1_score(true_labels, predictions, average='weighted')
        print(f"Test Loss: {running_loss/len(dataset_test)}, Accuracy: {running_acc/len(dataset_test)}")
        print(f"Precision: {precision}, Recall: {recall}, F1: {f1}")
        print(f"Class Counts: {class_counts}")
    print("Finished Testing")

    train_record = {"train_loss": train_loss, "train_acc": train_acc, "val_loss": val_loss, "val_acc": val_acc}
    test_record = {"true_labels": true_labels, "predictions": predictions, "precision": precision, "recall": recall, "f1": f1, "class_counts": class_counts}


    with open(f'{save_path}/train_record/train_record_lr{lr}_{rec_name}.pkl', 'wb') as f:
        pickle.dump(train_record, f)
    with open(f'{save_path}/test_record/test_record_lr{lr}_{rec_name}.pkl', 'wb') as f:
        pickle.dump(test_record, f)


Train Iter: 001/010  Loss: 3.1077: 100%|██████████| 1979/1979 [02:55<00:00, 11.27it/s]


Epoch 1, Loss: 3.4416097706803397, Accuracy: 0.11617844453217528


Valid Iter: 001/010  Loss: 2.2201: 100%|██████████| 15/15 [00:01<00:00, 11.09it/s]


Validation Loss: 3.011162149641249, Accuracy: 0.19111111111111112
Class Counts: [8, 6, 22, 11, 28, 2, 2, 6, 0, 3, 0, 5, 2, 7, 7, 13, 0, 11, 13, 1, 21, 2, 18, 3, 2, 6, 1, 1, 25, 3, 8, 3, 5, 23, 21, 3, 4, 11, 7, 21, 8, 7, 5, 17, 28, 6, 12, 3, 18, 11]


Train Iter: 002/010  Loss: 2.9394: 100%|██████████| 1979/1979 [02:55<00:00, 11.30it/s]


...

Epoch 10, Loss: 2.2926392355578904, Accuracy: 0.35976312672720095


Valid Iter: 010/010  Loss: 0.1628: 100%|██████████| 15/15 [00:01<00:00, 10.61it/s]


Validation Loss: 2.214882679051823, Accuracy: 0.3844444444444444
Class Counts: [4, 10, 13, 8, 12, 4, 6, 4, 5, 4, 5, 4, 5, 9, 8, 14, 7, 13, 5, 15, 12, 7, 7, 12, 12, 16, 4, 3, 9, 11, 5, 8, 7, 15, 7, 9, 13, 14, 9, 13, 12, 7, 3, 19, 7, 11, 8, 4, 14, 17]
Finished Training


Test Iter: 010/010  Loss: 1.6552: 100%|██████████| 15/15 [00:01<00:00, 10.71it/s]

Test Loss: 2.238057470851474, Accuracy: 0.38
Precision: 0.3902727827727828, Recall: 0.38, F1: 0.3635531224706753
Class Counts: [8, 11, 8, 5, 10, 15, 3, 7, 4, 2, 4, 4, 7, 14, 5, 10, 6, 9, 18, 14, 10, 3, 9, 15, 9, 10, 5, 7, 2, 12, 4, 6, 10, 21, 9, 12, 4, 11, 9, 10, 13, 13, 3, 18, 10, 15, 10, 9, 8, 9]
Finished Testing


In [21]:
with open('images/images.pickle', 'rb') as f:
    img_data = pickle.load(f)
class CustomDataset(Dataset):
    def __init__(self, img_data, txt_file, transform=None):
        self.data = img_data
        with open(txt_file, 'r') as f:
            self.labels = f.readlines()
        self.transform = transform
    def __len__(self):
        return len(self.labels)
    def __getitem__(self, idx):
        img_path, label = self.labels[idx].strip().split()
        img_path = img_path.split('/')
        img = self.data[img_path[1]][img_path[2]]
        img = Image.open(io.BytesIO(img)).convert('RGB')
        if self.transform:
            img = self.transform(img)
        label = torch.tensor(int(label))
        return img, label

transform_train = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.ColorJitter(brightness=0.1, contrast=0.2, saturation=0.2, hue=0),
    transforms.RandomRotation(degrees=10),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])


transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])
dataset_train = CustomDataset(img_data=img_data, txt_file='images/train.txt', transform=transform_train)
dataset_val = CustomDataset(img_data=img_data, txt_file='images/val.txt', transform=transform)
dataset_test = CustomDataset(img_data=img_data, txt_file='images/test.txt', transform=transform)


data_loader_train = DataLoader(dataset_train, batch_size=32, shuffle=True, num_workers=16)
data_loader_val = DataLoader(dataset_val, batch_size=32, shuffle=False, num_workers=16)
data_loader_test = DataLoader(dataset_test, batch_size=32, shuffle=False, num_workers=16)
if torch.cuda.is_available():
    device = torch.device("cuda")
else:
    device = torch.device("cpu")
class SelfAttention(nn.Module):
    def __init__(self, in_channels):
        super(SelfAttention, self).__init__()
        self.query_conv = nn.Conv2d(in_channels, in_channels // 8, kernel_size=1)
        self.key_conv = nn.Conv2d(in_channels, in_channels // 8, kernel_size=1)
        self.value_conv = nn.Conv2d(in_channels, in_channels, kernel_size=1)
        self.gamma = nn.Parameter(torch.zeros(1))
        
    def forward(self, x):
        batch_size, C, width, height = x.size()
        
        # 计算查询、键和值
        proj_query = self.query_conv(x).view(batch_size, -1, width * height).permute(0, 2, 1)
        proj_key = self.key_conv(x).view(batch_size, -1, width * height)
        energy = torch.bmm(proj_query, proj_key)
        
        # 计算注意力权重并应用到值上
        attention = F.softmax(energy, dim=-1)
        proj_value = self.value_conv(x).view(batch_size, -1, width * height)
        out = torch.bmm(proj_value, attention.permute(0, 2, 1))
        
        # 调整输出并加上残差连接
        out = out.view(batch_size, C, width, height)
        out = self.gamma * out + x
        return out

class CNNModel(nn.Module):
    def __init__(self, num_classes=50):
        super(CNNModel, self).__init__()
        self.conv1 = nn.Conv2d(3, 64, kernel_size=3, padding=1)
        self.bn1 = nn.BatchNorm2d(64)
        self.conv2 = nn.Conv2d(64, 128, kernel_size=3, padding=1)
        self.bn2 = nn.BatchNorm2d(128)
        self.conv3 = nn.Conv2d(128, 256, kernel_size=3, padding=1)
        self.bn3 = nn.BatchNorm2d(256)
        self.attention1 = SelfAttention(256)

        self.fc1 = nn.Linear(256*28*28, 2048)
        self.fc2 = nn.Linear(2048, num_classes)
        self.activation = nn.ReLU()
        self.flatten = nn.Flatten()
        self.pool = nn.MaxPool2d(2, 2)
        # self.dropout25 = nn.Dropout(0.25)
        self.dropout50 = nn.Dropout(0.5)
        
    def forward(self, x):
        x = self.pool(self.activation(self.bn1(self.conv1(x))))
        x = self.pool(self.activation(self.bn2(self.conv2(x))))
        x = self.pool(self.activation(self.bn3(self.conv3(x))))
        x = self.attention1(x)
        x = self.dropout50(self.flatten(x))
        x = self.dropout50(self.activation(self.fc1(x)))
        x = self.fc2(x)
        return x


for lr in [1e-5]:
    model = CNNModel(num_classes=50).to(device)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.AdamW(model.parameters(), lr=lr, weight_decay=0.00025)
    num_epochs = 10
    save_path = 'record/Q2_less_layers'
    rec_name = '6layers_2048_dropout50_adamW_0.00025_augmentless_round3'
    model_path = f'{save_path}/model_weight/model_weight_lr{lr}_{rec_name}.pth'
    best_val_loss = float('inf')
    best_model = None
    train_loss = []
    train_acc = []
    val_loss = []
    val_acc = []
    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0
        running_acc = 0.0
        pbar = tqdm(data_loader_train, colour='green', total=len(data_loader_train))
        for images, labels in pbar:
            images, labels = images.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()*len(labels)
            running_acc += (outputs.argmax(dim=1) == labels).float().sum().item()
            pbar.set_description(f'Train Iter: {epoch+1:03}/{num_epochs:03}  Loss: {loss:3.4f}')
            pbar.update()
        running_loss_avg = running_loss / len(dataset_train)
        running_acc_avg = running_acc / len(dataset_train)
        train_loss.append(running_loss_avg)
        train_acc.append(running_acc_avg)
        print(f"Epoch {epoch+1}, Loss: {running_loss_avg}, Accuracy: {running_acc_avg}")

        model.eval()
        with torch.no_grad():
            class_counts = [0] * 50
            running_loss = 0.0
            running_acc = 0.0
            pbar = tqdm(data_loader_val, colour='red', total=len(data_loader_val))
            for images, labels in pbar:
                images, labels = images.to(device), labels.to(device)
                outputs = model(images)
                predictions = outputs.argmax(dim=1)
                loss = criterion(outputs, labels)
                running_loss += loss.item()*len(labels)
                running_acc += (predictions == labels).float().sum().item()
                pbar.set_description(f'Valid Iter: {epoch+1:03}/{num_epochs:03}  Loss: {loss:3.4f}')
                pbar.update()

                for pred_label in predictions:
                    class_counts[pred_label] += 1

            running_loss_avg = running_loss / len(dataset_val)
            running_acc_avg = running_acc / len(dataset_val)
            val_loss.append(running_loss_avg)
            val_acc.append(running_acc_avg)
            print(f"Validation Loss: {running_loss_avg}, Accuracy: {running_acc_avg}")
            print(f"Class Counts: {class_counts}")
            if running_loss_avg < best_val_loss:
                best_val_loss = running_loss
                best_model = model
                torch.save(model.state_dict(), model_path)
        print("======================================")
    print("Finished Training")


    model.eval()
    with torch.no_grad():
        class_counts = [0] * 50
        running_loss = 0.0
        running_acc = 0.0
        predictions = []
        true_labels = []
        pbar = tqdm(data_loader_test, colour='red', total=len(data_loader_test))
        for images, labels in pbar:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)

            _, predicted = torch.max(outputs, 1)
            predictions.extend(predicted.cpu().numpy())
            true_labels.extend(labels.cpu().numpy())
            loss = criterion(outputs, labels)

            running_loss += loss.item()*len(labels)
            running_acc += (outputs.argmax(dim=1) == labels).float().sum().item()
            pbar.set_description(f'Test Iter: {epoch+1:03}/{num_epochs:03}  Loss: {loss:3.4f}')
            pbar.update()

        for pred_label in predictions:
            class_counts[pred_label] += 1
        
        precision = precision_score(true_labels, predictions, average='weighted')
        recall = recall_score(true_labels, predictions, average='weighted')
        f1 = f1_score(true_labels, predictions, average='weighted')
        print(f"Test Loss: {running_loss/len(dataset_test)}, Accuracy: {running_acc/len(dataset_test)}")
        print(f"Precision: {precision}, Recall: {recall}, F1: {f1}")
        print(f"Class Counts: {class_counts}")
    print("Finished Testing")

    train_record = {"train_loss": train_loss, "train_acc": train_acc, "val_loss": val_loss, "val_acc": val_acc}
    test_record = {"true_labels": true_labels, "predictions": predictions, "precision": precision, "recall": recall, "f1": f1, "class_counts": class_counts}


    with open(f'{save_path}/train_record/train_record_lr{lr}_{rec_name}.pkl', 'wb') as f:
        pickle.dump(train_record, f)
    with open(f'{save_path}/test_record/test_record_lr{lr}_{rec_name}.pkl', 'wb') as f:
        pickle.dump(test_record, f)


Train Iter: 001/010  Loss: 3.0541: 100%|██████████| 1979/1979 [02:55<00:00, 11.27it/s]


Epoch 1, Loss: 3.427572805647129, Accuracy: 0.11941571259376234


Valid Iter: 001/010  Loss: 1.8616: 100%|██████████| 15/15 [00:01<00:00, 10.70it/s]


Validation Loss: 2.992826567225986, Accuracy: 0.2111111111111111
Class Counts: [5, 17, 13, 8, 42, 9, 1, 4, 1, 0, 0, 0, 0, 2, 3, 9, 5, 3, 1, 0, 16, 14, 19, 7, 1, 13, 6, 13, 3, 7, 2, 7, 17, 34, 15, 5, 3, 4, 10, 23, 6, 10, 7, 14, 15, 8, 12, 5, 15, 16]


Train Iter: 002/010  Loss: 3.3663: 100%|██████████| 1979/1979 [02:55<00:00, 11.26it/s]


...

Epoch 10, Loss: 2.2907818063577077, Accuracy: 0.3630161863403079


Valid Iter: 010/010  Loss: 0.2965: 100%|██████████| 15/15 [00:01<00:00, 11.02it/s]


Validation Loss: 2.2528827528158826, Accuracy: 0.36666666666666664
Class Counts: [5, 6, 11, 10, 10, 10, 10, 9, 8, 2, 6, 2, 4, 8, 6, 3, 4, 10, 16, 24, 13, 10, 8, 7, 12, 14, 1, 13, 10, 4, 4, 8, 7, 10, 9, 10, 11, 11, 11, 11, 15, 9, 4, 12, 9, 8, 10, 12, 12, 11]
Finished Training


Test Iter: 010/010  Loss: 0.7742: 100%|██████████| 15/15 [00:01<00:00, 10.92it/s]


Test Loss: 2.217808720800612, Accuracy: 0.38666666666666666
Precision: 0.39547956945015766, Recall: 0.38666666666666666, F1: 0.3733428399327847
Class Counts: [6, 5, 14, 12, 6, 12, 11, 12, 6, 2, 5, 2, 6, 16, 12, 4, 5, 9, 18, 11, 11, 7, 11, 9, 9, 11, 5, 6, 11, 8, 2, 5, 10, 17, 13, 9, 4, 8, 11, 9, 7, 8, 8, 11, 10, 13, 14, 13, 8, 8]
Finished Testing


In [4]:
with open('images/images.pickle', 'rb') as f:
    img_data = pickle.load(f)
class CustomDataset(Dataset):
    def __init__(self, img_data, txt_file, transform=None):
        self.data = img_data
        with open(txt_file, 'r') as f:
            self.labels = f.readlines()
        self.transform = transform
    def __len__(self):
        return len(self.labels)
    def __getitem__(self, idx):
        img_path, label = self.labels[idx].strip().split()
        img_path = img_path.split('/')
        img = self.data[img_path[1]][img_path[2]]
        img = Image.open(io.BytesIO(img)).convert('RGB')
        if self.transform:
            img = self.transform(img)
        label = torch.tensor(int(label))
        return img, label

transform_train = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.ColorJitter(brightness=0.1, contrast=0.2, saturation=0.2, hue=0),
    transforms.RandomRotation(degrees=10),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])


transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])
dataset_train = CustomDataset(img_data=img_data, txt_file='images/train.txt', transform=transform_train)
dataset_val = CustomDataset(img_data=img_data, txt_file='images/val.txt', transform=transform)
dataset_test = CustomDataset(img_data=img_data, txt_file='images/test.txt', transform=transform)


data_loader_train = DataLoader(dataset_train, batch_size=32, shuffle=True, num_workers=16)
data_loader_val = DataLoader(dataset_val, batch_size=32, shuffle=False, num_workers=16)
data_loader_test = DataLoader(dataset_test, batch_size=32, shuffle=False, num_workers=16)
if torch.cuda.is_available():
    device = torch.device("cuda")
else:
    device = torch.device("cpu")


class CNNModel(nn.Module):
    def __init__(self, num_classes=50):
        super(CNNModel, self).__init__()
        self.conv1 = nn.Conv2d(3, 64, kernel_size=3, padding=1)
        self.bn1 = nn.BatchNorm2d(64)
        self.conv2 = nn.Conv2d(64, 128, kernel_size=3, padding=1)
        self.bn2 = nn.BatchNorm2d(128)
        self.conv3 = nn.Conv2d(128, 256, kernel_size=3, padding=1)
        self.bn3 = nn.BatchNorm2d(256)
        # self.attention1 = SelfAttention(256)

        self.fc1 = nn.Linear(256*28*28, 2048)
        self.fc2 = nn.Linear(2048, num_classes)
        self.activation = nn.ReLU()
        self.flatten = nn.Flatten()
        self.pool = nn.MaxPool2d(2, 2)
        # self.dropout25 = nn.Dropout(0.25)
        self.dropout50 = nn.Dropout(0.5)
        
    def forward(self, x):
        x = self.pool(self.activation(self.bn1(self.conv1(x))))
        x = self.pool(self.activation(self.bn2(self.conv2(x))))
        x = self.pool(self.activation(self.bn3(self.conv3(x))))
        x = self.dropout50(self.flatten(x))
        x = self.dropout50(self.activation(self.fc1(x)))
        x = self.fc2(x)
        return x


for lr in [1e-5]:
    model = CNNModel(num_classes=50).to(device)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.AdamW(model.parameters(), lr=lr, weight_decay=0.00025)
    num_epochs = 10
    save_path = 'record/Q2_less_layers'
    rec_name = '5layers_2048_dropout50_adamW_0.00025_augmentless_noattention'
    model_path = f'{save_path}/model_weight/model_weight_lr{lr}_{rec_name}.pth'
    best_val_loss = float('inf')
    best_model = None
    train_loss = []
    train_acc = []
    val_loss = []
    val_acc = []
    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0
        running_acc = 0.0
        pbar = tqdm(data_loader_train, colour='green', total=len(data_loader_train))
        for images, labels in pbar:
            images, labels = images.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()*len(labels)
            running_acc += (outputs.argmax(dim=1) == labels).float().sum().item()
            pbar.set_description(f'Train Iter: {epoch+1:03}/{num_epochs:03}  Loss: {loss:3.4f}')
            pbar.update()
        running_loss_avg = running_loss / len(dataset_train)
        running_acc_avg = running_acc / len(dataset_train)
        train_loss.append(running_loss_avg)
        train_acc.append(running_acc_avg)
        print(f"Epoch {epoch+1}, Loss: {running_loss_avg}, Accuracy: {running_acc_avg}")

        model.eval()
        with torch.no_grad():
            class_counts = [0] * 50
            running_loss = 0.0
            running_acc = 0.0
            pbar = tqdm(data_loader_val, colour='red', total=len(data_loader_val))
            for images, labels in pbar:
                images, labels = images.to(device), labels.to(device)
                outputs = model(images)
                predictions = outputs.argmax(dim=1)
                loss = criterion(outputs, labels)
                running_loss += loss.item()*len(labels)
                running_acc += (predictions == labels).float().sum().item()
                pbar.set_description(f'Valid Iter: {epoch+1:03}/{num_epochs:03}  Loss: {loss:3.4f}')
                pbar.update()

                for pred_label in predictions:
                    class_counts[pred_label] += 1

            running_loss_avg = running_loss / len(dataset_val)
            running_acc_avg = running_acc / len(dataset_val)
            val_loss.append(running_loss_avg)
            val_acc.append(running_acc_avg)
            print(f"Validation Loss: {running_loss_avg}, Accuracy: {running_acc_avg}")
            print(f"Class Counts: {class_counts}")
            if running_loss_avg < best_val_loss:
                best_val_loss = running_loss
                best_model = model
                torch.save(model.state_dict(), model_path)
        print("======================================")
    print("Finished Training")


    model.eval()
    with torch.no_grad():
        class_counts = [0] * 50
        running_loss = 0.0
        running_acc = 0.0
        predictions = []
        true_labels = []
        pbar = tqdm(data_loader_test, colour='red', total=len(data_loader_test))
        for images, labels in pbar:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)

            _, predicted = torch.max(outputs, 1)
            predictions.extend(predicted.cpu().numpy())
            true_labels.extend(labels.cpu().numpy())
            loss = criterion(outputs, labels)

            running_loss += loss.item()*len(labels)
            running_acc += (outputs.argmax(dim=1) == labels).float().sum().item()
            pbar.set_description(f'Test Iter: {epoch+1:03}/{num_epochs:03}  Loss: {loss:3.4f}')
            pbar.update()

        for pred_label in predictions:
            class_counts[pred_label] += 1
        
        precision = precision_score(true_labels, predictions, average='weighted')
        recall = recall_score(true_labels, predictions, average='weighted')
        f1 = f1_score(true_labels, predictions, average='weighted')
        print(f"Test Loss: {running_loss/len(dataset_test)}, Accuracy: {running_acc/len(dataset_test)}")
        print(f"Precision: {precision}, Recall: {recall}, F1: {f1}")
        print(f"Class Counts: {class_counts}")
    print("Finished Testing")

    train_record = {"train_loss": train_loss, "train_acc": train_acc, "val_loss": val_loss, "val_acc": val_acc}
    test_record = {"true_labels": true_labels, "predictions": predictions, "precision": precision, "recall": recall, "f1": f1, "class_counts": class_counts}


    with open(f'{save_path}/train_record/train_record_lr{lr}_{rec_name}.pkl', 'wb') as f:
        pickle.dump(train_record, f)
    with open(f'{save_path}/test_record/test_record_lr{lr}_{rec_name}.pkl', 'wb') as f:
        pickle.dump(test_record, f)


Train Iter: 001/010  Loss: 3.6908: 100%|██████████| 1979/1979 [02:47<00:00, 11.81it/s]


Epoch 1, Loss: 3.439309311820451, Accuracy: 0.115499407816818


Valid Iter: 001/010  Loss: 2.6167: 100%|██████████| 15/15 [00:01<00:00, 13.41it/s]


Validation Loss: 3.0437215614318847, Accuracy: 0.18444444444444444
Class Counts: [10, 0, 35, 4, 49, 15, 4, 7, 0, 0, 0, 0, 0, 8, 5, 11, 0, 1, 1, 2, 11, 10, 10, 6, 6, 10, 0, 3, 4, 4, 14, 2, 9, 29, 31, 7, 2, 16, 5, 18, 10, 6, 5, 23, 11, 5, 15, 3, 18, 5]


Train Iter: 002/010  Loss: 2.9658: 100%|██████████| 1979/1979 [02:47<00:00, 11.78it/s]


...

Epoch 10, Loss: 2.52874028566935, Accuracy: 0.30185550730359256


Valid Iter: 010/010  Loss: 0.4356: 100%|██████████| 15/15 [00:01<00:00, 12.63it/s]


Validation Loss: 2.4062459495332504, Accuracy: 0.31777777777777777
Class Counts: [9, 9, 9, 7, 16, 8, 8, 7, 7, 2, 1, 0, 4, 14, 24, 5, 7, 10, 4, 18, 6, 7, 17, 4, 10, 10, 5, 12, 7, 7, 3, 2, 10, 8, 7, 14, 8, 12, 15, 20, 12, 2, 13, 12, 5, 9, 8, 5, 14, 17]
Finished Training


Test Iter: 010/010  Loss: 1.1154: 100%|██████████| 15/15 [00:01<00:00, 12.51it/s]


Test Loss: 2.359922768274943, Accuracy: 0.35555555555555557
Precision: 0.38631091122499794, Recall: 0.35555555555555557, F1: 0.3412440434481002
Class Counts: [11, 8, 8, 9, 1, 17, 7, 12, 8, 2, 3, 6, 4, 13, 20, 12, 3, 9, 8, 16, 7, 3, 13, 6, 5, 11, 6, 12, 5, 5, 3, 2, 8, 15, 12, 8, 6, 5, 19, 12, 15, 6, 15, 9, 9, 13, 9, 12, 6, 16]
Finished Testing
